# Lecture et traitement des données issues des thermo-mosaïques

In [ ]:
import os
import  matplotlib.pyplot as plt
from matplotlib import cm
import geopandas as gpd
import pandas as pd
import rasterio as rio
from rasterio.plot import plotting_extent
from rasterio.plot import show
from rasterio.plot import show_hist # Useful if you wish to plot all hist and GPS target image
from rasterio.mask import mask
from rasterio.enums import Resampling
from shapely.geometry.point import Point
import shapely
import numpy as np
import csv
import fiona
from tools_AA_IR import reading_gps_file, circle_sensor, plottingtemp_single_label_IR
from tools_AA_IR import path_IR, get_tif,get_value_IR
from tools_AA_IR import circle_to_shape, readingIR , IR_mask, readingIR_all,alternative_readingIR_all
from tools_AA import readingtemp_AA, slice_raw, path_sonde, name_sonde, plottingtemp_single_label
from tools_EL import readingtemp, plottingtemp
from tools_AA_variograms import reading_3band, reading_cluster,get_3band
from tools_AA_variograms import reshape_3band_to_dataframe, set_bound_to_NAN
from matplotlib.gridspec import GridSpec
from sklearn.metrics import mean_squared_error
from sklearn.linear_model import Ridge
from sklearn.preprocessing import PolynomialFeatures
from sklearn.pipeline import make_pipeline
import seaborn as sns
from scipy.interpolate import interp1d

In [ ]:
# Ouverture et recupération des positions des sondes
filename_Sensor_txt = "./traitement_PIREN/sondes_gps_UTM31N_phase1.txt"
Sensor_coord = reading_gps_file(filename_Sensor_txt)
Sensor_coord # Contient les coord de toutes les sondes

## Importation des données

In [ ]:
#Recupere les noms des IR dans le dossier path, 
#par defaut = './traitement_PIREN/' et permet de 
#choisir n'importe quelle ortho avec son nom 

filetif = ['6H55','7H29','8H22','9H28','10H22','11H27','12H31','13H26','15H59','17H27']

#filetif = ['6H55']


ls_path_tif,filetif = get_tif(filetif)
dict_IR= alternative_readingIR_all(ls_path_tif[::2],filetif)

## Plot de toutes les sondes/IR

# Fenêtrage des sondes, création de Dict_IR

In [ ]:
from rasterio.windows import Window
def reading_1band(LONGUEUR,filetif,ls_path_tif,LABEL = "band 1") :
    """ Lit un fichier .tif à 1 band et retourne un patch carré de coté LONGUEUR au format DataFrame [x,y,band 1]
    """
    
    #filename = './traitement_PIREN/vis_piren_phase_HSV.tif'
    #LONGUEUR = 10
    filename_Sensor_txt = "./traitement_PIREN/sondes_gps_UTM31N_phase1.txt"
    sensor_coord = reading_gps_file(filename_Sensor_txt)
    dict_IR = {} # va contenir les noms IR + tous les dicts associés à chaque sonde
    
    for m,tif in enumerate(ls_path_tif) : 
        with rio.open(tif) as dataset :
            #Value par défaut
            #win_height = 6303
            #win_width = 3421
            

            ls_index_target = [] # liste des valeurs indexées dans l'image
            win_height = round(LONGUEUR / dataset.res[0])
            win_width  = round(LONGUEUR / dataset.res[1])
            print("Taille de la fenetre :",win_height,"x",win_width)
            print("Résolution ","x:",dataset.res[0],"y:",dataset.res[1])

            # Loop pour recuperer les coordonnées UTM et convertir en indice 
            for k in range(len(sensor_coord["SensorName"])) :
                x = sensor_coord["x"][k] 
                y = sensor_coord["y"][k]
                target = dataset.index(float(x),float(y))
                ls_index_target.append(target)

            dict_windows = {} # liste des paramètres dans la création d'une window
            dict_windows_param = {} # dict des targets : améliore la lisibilité

            for i,index_target in enumerate(ls_index_target) :
                win = Window.from_slices((index_target[0]-(win_height//2),(index_target[0]+(win_height//2))),
                                         (index_target[1]-(win_width//2),(index_target[1]+(win_width//2)))
                                         )
                win_transform = dataset.window_transform(win)
                all_band = dataset.read(1,window = win) # en cas de visualisation

                dict_param = {"win" : win,
                                "win_transform" : win_transform,
                               "all_band" : all_band,
                               "SensorName" : sensor_coord["SensorName"][i],
                             "LONGUEUR" : LONGUEUR,
                             "resolution" : dataset.res}

                dict_windows_param[sensor_coord["SensorName"][i]] = dict_param

                # Vecteurs linéaires avec valeurs uniforméments crées
                x_start = win_transform[2]
                x_res   = win_transform[0]
                x_end   = x_start+(float(x_res)*win.width)

                y_end   = win_transform[5]
                y_res   = win_transform[4]
                y_start = y_end + (float(y_res)*win.height)

                #print("x_end =",x_end)
                #print("y_start =",y_start)

                x = np.linspace(x_start,x_end,num = win.width+1)
                y = np.linspace(y_start,y_end,num = win.height+1)

                band_1 = []
                x_matrix = []
                y_matrix = []
                # Loop pour obtenir la valeur de chaque pt pour chaque band
                
                for j in range(len(x)) :
                    for l in range(len(y)) :
                        for val in dataset.sample([(x[j],y[l])]): 
                            band_1.append(val[0])
                            x_matrix.append(x[j]) # permet de répeter le terme 
                            y_matrix.append(y[l])
                mapping = ['x','y',LABEL]
                DATA_WINDOW = pd.DataFrame(np.array([x_matrix,
                                                     y_matrix,
                                                     band_1],dtype = object).T,
                                           columns=mapping)

                DATA_WINDOW['SensorName'] = sensor_coord["SensorName"][i]

                dict_windows[sensor_coord["SensorName"][i]] = DATA_WINDOW
        dict_IR["IR_"+filetif[m]] = {"dict_windows" : dict_windows,
                                     "dict_windows_param" : dict_windows_param}
        print("complited : ","IR_"+filetif[m])
         
    return dict_IR  

# Création de Dict_sensor

### Normalisation des données par ajout de bande à un DataFrame par Sonde/IR

In [ ]:
def sensor_to_DataFrame(dict_IR):
    """
    Création d'un DataFrame par sonde contenant les positions et les des bandes normalisées, réunit sous 
    un dictionnaire
    """
    S1 = pd.DataFrame(columns = ["x","y","origin band","band 1","band N","IR"], dtype = float)
    S2 = pd.DataFrame(columns = ["x","y","origin band","band 1","band N","IR"], dtype = float)
    S3_AIR = pd.DataFrame(columns = ["x","y","origin band","band 1","band N","IR"], dtype = float)
    S4 = pd.DataFrame(columns = ["x","y","origin band","band 1","band N","IR"], dtype = float)
    S5 = pd.DataFrame(columns = ["x","y","origin band","band 1","band N","IR"], dtype = float)
    S6 = pd.DataFrame(columns = ["x","y","origin band","band 1","band N","IR"], dtype = float)
    S7 = pd.DataFrame(columns = ["x","y","origin band","band 1","band N","IR"], dtype = float)
    S8 = pd.DataFrame(columns = ["x","y","origin band","band 1","band N","IR"], dtype = float)
    S9 = pd.DataFrame(columns = ["x","y","origin band","band 1","band N","IR"], dtype = float)
    S10 = pd.DataFrame(columns = ["x","y","origin band","band 1","band N","IR"], dtype = float)
    dict_sensor = {"S1" : S1,"S2" : S2,"S3-AIR" : S3_AIR,"S4" : S4,"S5" : S5,
                "S6" : S6,"S7" : S7,"S8" : S8,"S9" : S9,"S10" : S10}



    for sensor in dict_sensor :
        k = 0
        BAND_MIN_N = 0
        BAND_MAX_N = 0
        for target in dict_IR :
            min_= dict_IR[target]["dict_windows"][sensor]["band 1"].min()
            max_= dict_IR[target]["dict_windows"][sensor]["band 1"].max()
            if min_ < BAND_MIN_N :
                BAND_MIN_N = min_


            if max_ > BAND_MAX_N :
                BAND_MAX_N = max_


        print("BAND_MIN_N",BAND_MIN_N) # valeur minimale retenue pour la normalisation
        print("BAND_MAX_N",BAND_MAX_N) # valeur maximale retenue pour la normalisation

        for target in dict_IR :

            band_shape = dict_IR[target]["dict_windows"][sensor][["band 1"]].shape[0]

            band_min = dict_IR[target]["dict_windows"][sensor]["band 1"].min()
            band_max = dict_IR[target]["dict_windows"][sensor]["band 1"].max()

            x = pd.DataFrame(dict_IR[target]["dict_windows"][sensor]["x"], dtype = float)
            y = pd.DataFrame(dict_IR[target]["dict_windows"][sensor]["y"], dtype = float)
            # band originale, sans normalisation
            origin_band = pd.DataFrame(dict_IR[target]["dict_windows"][sensor]["band 1"], dtype = float)
            origin_band.columns = ["origin band"]

            # band avec normalisation différente pour chaque IR
            band     = pd.DataFrame((dict_IR[target]["dict_windows"][sensor][["band 1"]]-band_min)/(band_max-band_min), dtype = float)

            # band avec normalisation commune pour chaque IR
            band_N   = pd.DataFrame((dict_IR[target]["dict_windows"][sensor][["band 1"]]-BAND_MIN_N)/(BAND_MAX_N-BAND_MIN_N), dtype = float) 
            band_N.columns = ['band N']

            # Concaténation de plusieurs bandes
            cnte = pd.concat([x,y,origin_band,band,band_N],axis=1)
            dict_sensor[sensor]=dict_sensor[sensor].append(cnte,ignore_index=True)

            # Repetition du nom de IR pour chaque valeurs -> boxplot
            dict_sensor[sensor]["IR"][k:(k+band_shape)]  = target

            k+=band_shape
        print("completed :",sensor)
    return dict_sensor


In [ ]:
filetif

In [ ]:
filetif = ['6H55','7H29','8H22','9H28','10H22','11H27','12H31','13H26',
                               '15H59','17H27']

#filetif = ['6H55']
ls_path_tif,filetif = get_tif(filetif)
Dimension = 4 # en m
dict_IR = reading_1band(Dimension,filetif,ls_path_tif)
dict_sensor = sensor_to_DataFrame(dict_IR)

## Moyenne entre deux rayon de cercle r1 et r2

In [ ]:
def get_value_pseudo_temp(ls_path_tif,dict_IR,r1 = 0.7 , r2 = 1.5) :
    """
    Création d'un dictionnaire classé par IR contenant toutes les données comprise entre un rayon r1 et r2, 
    """
    
    dict_value_IR = {}

    for i,IR in enumerate(dict_IR) :
        dict_value_sensor = {}
        with rio.open(ls_path_tif[i]) as dataset : # ouverture du fichier obligatoire pour recup la valeur
            for sensor in dict_IR[IR]["dict_windows"] :

                res = dict_IR[IR]["dict_windows_param"][sensor]["resolution"] # resolution [x,y]
                sensor_dataframe = dict_IR[IR]["dict_windows"][sensor]
                mid_indice = (dict_IR[IR]["dict_windows_param"][sensor]['win'].width) # les indices commencent à 0 !

                mid_x = sensor_dataframe.iloc[len(sensor_dataframe)//2]["x"] 
                mid_y = sensor_dataframe.iloc[(mid_indice//2)-1]["y"]# la valeur du milieu est //2 

                #list des valeurs autour de [0, 2pi] , mouvement anti-horaire
                theta = np.linspace(0,np.pi*2,round(np.pi*2/res[0]))
                ls_x = np.empty([1]) ; ls_y = np.empty([1])

                #recherche de points entre deux rayon r1 et r2
                
                for THETA in theta :
                    #
                    x1 = mid_x + r1*np.cos(THETA)
                    y1 = mid_y + r1*np.sin(THETA)

                    x2 = mid_x + r2*np.cos(THETA)
                    y2 = mid_y + r2*np.sin(THETA)

                    ls_x = np.append(ls_x,np.linspace(x1,x2,round((r2-r1)/res[0]))) 
                    ls_y = np.append(ls_y,np.linspace(y1,y2,round((r2-r1)/res[1])))

                ls_x = np.delete(ls_x,0) ; ls_y = np.delete(ls_y,0)


                with rio.open(ls_path_tif[i]) as dataset : # ouverture du fichier obligatoire pour recup la valeur

                    dict_IR[IR]["dict_windows_param"][sensor]['win'].width # résolution -> nb de pixel == nb d'indice 

                    ls_value = []
                    for k in range(len(ls_x)) :

                        for value in dataset.sample([(float(ls_x[k]),float(ls_y[k]))]) : 
                            ls_value.append(float(value))

                    dict_value_sensor[sensor] = pd.DataFrame(np.array([ls_x,ls_y,ls_value],dtype = float).T,columns = 
                                                             ["x","y","value"])
        print(IR)
        dict_value_IR[IR] = dict_value_sensor
        print("nombre de coordonnées utilisée pour la moyenne :",dict_value_IR[IR][sensor].shape[0])
        
    return dict_value_IR

In [ ]:
#importation des chemins d'accès aux IRs
filetif = ['6H55','7H29','8H22','9H28','10H22','11H27','12H31','13H26',
                               '15H59','17H27']

#filetif = ["6H55"]
ls_path_tif,filetif = get_tif(filetif)

# dictionnaire contenant les valeurs comprise entre les deux rayons de recherche
dict_value_IR = get_value_pseudo_temp(ls_path_tif,dict_IR)

## Ajout des fenêtres de clustering 

In [ ]:
def reading_cluster_modified(LONGUEUR,filename='./traitement_PIREN/vis_piren_phase_HSV.tif', x = None,y = None) :
    """ Lit un fichier .tif à 3 bands et retourne un patch carré de coté LONGUEUR au format csv pour une certaine coordonnée
    x,y
    """
    #filename = './traitement_PIREN/vis_piren_phase_HSV.tif'
    #LONGUEUR = 10
    filename_Sensor_txt = "./traitement_PIREN/sondes_gps_UTM31N_phase1.txt"
    sensor_coord = reading_gps_file(filename_Sensor_txt)
    with rio.open(filename) as dataset :
        #Value par défaut
        #win_height = 6303
        #win_width = 3421

        ls_index_target = [] # liste des valeurs indexées dans l'image
        win_height = round(LONGUEUR / dataset.res[0])
        win_width  = round(LONGUEUR / dataset.res[1])
        print("Taille de la fenetre :",win_height,"x",win_width)

        # Loop pour recuperer les coordonnées UTM et convertir en indice 
        if x != None or y != None :
            ls_index_target.append(dataset.index(float(x),float(y)))
        else :
            
            for k in range(len(sensor_coord["SensorName"])) :
                x = sensor_coord["x"][k] 
                y = sensor_coord["y"][k]
                target = dataset.index(float(x),float(y))
                ls_index_target.append(target)

        dict_windows = {} # liste des paramètres dans la création d'une window
        dict_windows_param = {} # dict des targets : améliore la lisibilité

        for i,index_target in enumerate(ls_index_target) :
            win = Window.from_slices((index_target[0]-(win_height//2),(index_target[0]+(win_height//2))),
                                     (index_target[1]-(win_width//2),(index_target[1]+(win_width//2)))
                                     )
            
            win_transform = dataset.window_transform(win)
            all_band = dataset.read(1,window = win) # en cas de visualisation

            dict_param = {"win" : win,
                            "win_transform" : win_transform,
                           "all_band" : all_band,
                           "SensorName" : sensor_coord["SensorName"][i]}

            dict_windows_param = dict_param

            # Vecteurs linéaires avec valeurs uniforméments crées
            x_start = win_transform[2]
            x_res   = win_transform[0]
            x_end   = x_start+(float(x_res)*win.width)

            y_end   = win_transform[5]
            y_res   = win_transform[4]
            y_start = y_end + (float(y_res)*win.height)

            #print("x_end =",x_end)
            #print("y_start =",y_start)

            x = np.linspace(x_start,x_end,num = win.width)
            y = np.linspace(y_start,y_end,num = win.height)

            band_1 = []
            x_matrix = []
            y_matrix = []
            # Loop pour obtenir la valeur de chaque pt pour chaque band
            for j in range(len(x)) :
                for l in range(len(y)) :
                    for val in dataset.sample([(x[j],y[l])]): 
                        label = val[0]
                        band_1.append(label)
                        x_matrix.append(x[j]) # permet de répeter le terme 
                        y_matrix.append(y[l])
                        mapping = ['x','y','label']
            DATA_WINDOW = pd.DataFrame(np.array([x_matrix,
                                                 y_matrix,
                                                 band_1],dtype = object).T,
                                       columns=mapping)
            
         
            
    return DATA_WINDOW, dict_windows_param

In [ ]:
def reading_3band_modified(LONGUEUR,filename='./traitement_PIREN/vis_piren_phase_HSV.tif',normalization = True, x = None,y = None) :
    """ Lit un fichier .tif à 3 bands et retourne un patch carré de coté LONGUEUR au format csv pour l'execution d'un variogram
    """
    #filename = './traitement_PIREN/vis_piren_phase_HSV.tif'
    #LONGUEUR = 10
    filename_Sensor_txt = "./traitement_PIREN/sondes_gps_UTM31N_phase1.txt"
    sensor_coord = reading_gps_file(filename_Sensor_txt)
    with rio.open(filename) as dataset :
        #Value par défaut
        #win_height = 6303
        #win_width = 3421

        ls_index_target = [] # liste des valeurs indexées dans l'image
        win_height = round(LONGUEUR / dataset.res[0])
        win_width  = round(LONGUEUR / dataset.res[1])
        print("Taille de la fenetre :",win_height,"x",win_width)

        # Loop pour recuperer les coordonnées UTM et convertir en indice 
        if x != None or y != None :
            
            ls_index_target.append(dataset.index(float(x),float(y)))
            
        else :
            
            for k in range(len(sensor_coord["SensorName"])) :
                x = sensor_coord["x"][k] 
                y = sensor_coord["y"][k]
                target = dataset.index(float(x),float(y))
                ls_index_target.append(target)

        dict_windows = {} # liste des paramètres dans la création d'une window
        dict_windows_param = {} # dict des targets : améliore la lisibilité

        for i,index_target in enumerate(ls_index_target) :
            win = Window.from_slices((index_target[0]-(win_height//2),(index_target[0]+(win_height//2))),
                                     (index_target[1]-(win_width//2),(index_target[1]+(win_width//2)))
                                     )
            win_transform = dataset.window_transform(win)
            all_band = dataset.read([1,2,3],window = win) # en cas de visualisation

            dict_param = {"win" : win,
                            "win_transform" : win_transform,
                           "all_band" : all_band,
                           "SensorName" : sensor_coord["SensorName"][i]}

            dict_windows_param[sensor_coord["SensorName"][i]] = dict_param

            # Vecteurs linéaires avec valeurs uniforméments crées
            x_start = win_transform[2]
            x_res   = win_transform[0]
            x_end   = x_start+(float(x_res)*win.width)

            y_end   = win_transform[5]
            y_res   = win_transform[4]
            y_start = y_end + (float(y_res)*win.height)

            #print("x_end =",x_end)
            #print("y_start =",y_start)

            x = np.linspace(x_start,x_end,num = win.width)
            y = np.linspace(y_start,y_end,num = win.height)

            band_1 = []
            band_2 = []
            band_3 = []
            band_RGB = []
            band_greeness =[]
            x_matrix = []
            y_matrix = []
            # Loop pour obtenir la valeur de chaque pt pour chaque band
            if normalization == False :
                for j in range(len(x)) :
                    for l in range(len(y)) :
                        for val in dataset.sample([(x[j],y[l])]): 
                            R = val[0]
                            G = val[1]
                            B = val[2]
                            band_1.append(R)
                            band_2.append(G)
                            band_3.append(B)
                            x_matrix.append(x[j]) # permet de répeter le terme 
                            y_matrix.append(y[l])
                            mapping = ['x','y','band 1','band 2','band 3']
                DATA_WINDOW = pd.DataFrame(np.array([x_matrix,
                                                     y_matrix,
                                                     band_1,
                                                     band_2,
                                                     band_3],dtype = object).T,
                                           columns=mapping)
                        
            else :
                for j in range(len(x)) :
                    for l in range(len(y)) :
                        for val in dataset.sample([(x[j],y[l])]): 
                            R = val[0]
                            G = val[1]
                            B = val[2]
                            band_1.append(R)
                            band_2.append(G)
                            band_3.append(B)
                            RGB = np.sqrt(R**2 + G**2 + B**2)
                            Greeness = G/(R+G+B)
                            band_RGB.append(RGB)
                            band_greeness.append(Greeness)
                            x_matrix.append(x[j]) # permet de répeter le terme 
                            y_matrix.append(y[l])
                            
                mapping = ['x','y','band 1',
                           'band 2','band 3',
                           'band RGB','band Greensess']
                print(len(band_greeness))
                DATA_WINDOW = pd.DataFrame(np.array([x_matrix,
                                                     y_matrix,
                                                     band_1,
                                                     band_2,
                                                     band_3,
                                                    band_RGB,
                                                     band_greeness],
                                                    dtype = object).T,
                                           columns=mapping)
   
            DATA_WINDOW['SensorName'] = sensor_coord["SensorName"][i]
            
         
            
    return dict_param, DATA_WINDOW  

#### Visible

In [ ]:
# Fichier .tif contenant les clusters 
fileout_cluster_vis = "./traitement_PIREN/vis_piren_phase_1_cropped_clustered_7k.tif"
print("file cluster :",fileout_cluster_vis)

dict_windows_cluster, dict_windows_param_cluster = reading_cluster(10,fileout_cluster_vis)#,x = 523630,y = 5366180)

# Fichier original 

filename_vis = './traitement_PIREN/vis_piren_phase1_ortho_UTM31N.tif'
print("file 3band :",filename_vis)
dict_windows, dict_windows_param = reading_3band(10,filename_vis,False)#,x = 523630,y = 5366180)

##### ajout des labels de clusterings pour box plot

In [ ]:
for sensor in dict_windows : 
    dict_windows[sensor]["clustering_label"] = dict_windows_cluster[sensor]["label"]
    

#### HSV

In [ ]:
# Fichier .tif contenant les clusters 
fileout_cluster_hsv = "./traitement_PIREN/vis_piren_phase_1_cropped_HSV_clustered_7k.tif"
print("file cluster :",fileout_cluster_hsv)

dict_windows_cluster_hsv, dict_windows_param_cluster_hsv = reading_cluster(10,fileout_cluster_hsv)#,x = 523630,y = 5366180)

# Fichier original 

filename_hsv = "./traitement_PIREN/vis_piren_phase_1_cropped_HSV.tif"
print("file 3band :",filename_hsv)
dict_windows_hsv, dict_windows_param_hsv = reading_3band(10,filename_hsv,False)

##### ajout des labels de clusterings pour box plot

In [ ]:
for sensor in dict_windows_hsv : 
    dict_windows_hsv[sensor]["clustering_label"] = dict_windows_cluster_hsv[sensor]["label"]
    

# Ajout des températures pour une date donnée en DataFrame 

In [ ]:
### A MODIFIER 
date_1='2019-04-14 06:00:00'
date_2='2019-04-14 18:30:00'


In [ ]:
def dict_temp_sensor_function(date_1,date_2) :
    """
    Dictionnaire regroupant les DATAFRAMES tronquées aux dates souhaitées 
    """
    
    ls_path = path_sonde()
    dict_temp_sensor = {}
    for i,ls_path_sonde in enumerate(ls_path):
        name = ls_path_sonde[0][ls_path_sonde[0].find("S"):(ls_path_sonde[0].find(".csv"))]
        Raw = readingtemp(ls_path_sonde[0])
        if name == "S3AIR" : #différence entre le nom du fichier et le reste du code.
            name = "S3-AIR"
        dict_temp_sensor[name] = slice_raw(Raw,date_1,date_2)
        print("completed :",name)
    
    return dict_temp_sensor

    

In [ ]:
dict_temp_sensor = dict_temp_sensor_function(date_1,date_2)

### Ajout des Pseudos-Températures

In [ ]:
def evaluate_pseudo_temp(dict_value_IR) :
    """
    Fonction qui permet d'établir les valeurs de Pseudo-température (Signal IR aux emplacements des sondes)
    """
    
    requested_sonde = ['S1', 'S10', 'S2', 'S3-AIR', 'S4', 'S5', 'S6', 'S7', 'S8', 'S9']
    pseudo_temp = {}
    for sonde in requested_sonde :
        pseudo_temp_IR = {}
        for IR in dict_value_IR :

            var = dict_value_IR[IR][sonde]["value"].std()
            mean = dict_value_IR[IR][sonde]["value"].mean()
            print(sonde,IR,var,"mean :",mean)
            pseudo_temp_IR[IR] = {"values" : mean,
                                  "var" : var,
                                  "time" : np.NAN }
            print("var :",pseudo_temp_IR[IR]['var'])
            pseudo_temp_IR[IR]['time'] = pd.Timestamp(str('2019-04-14' +' '
                                                          +IR[IR.find('_')+1:IR.find(('H'))] + ':' + 
                                                          IR[IR.find('H')+1::] )).round('30min')
        
        pseudo_temp[sonde] = pseudo_temp_IR
        
    return pseudo_temp

In [ ]:
pseudo_temp = evaluate_pseudo_temp(dict_value_IR)

### Localisation des sondes

In [ ]:
filename_Sensor_txt = "./traitement_PIREN/sondes_gps_UTM31N_phase1.txt"
sensor_coord = reading_gps_file(filename_Sensor_txt)


In [ ]:
def plottingtemp_single_label_IR_a(Raw,fig,ax1,label,step,random_color = np.random.randint(10)):
    """
    Plot d'un seul capteur (T1,..,T4) pour toutes les sondes
    """
    coloration=plt.cm.Set1(np.linspace(0,1,10))
    
    dict_label = {'T1' :'T1-0.50','T2' :'T2-0.35','T3' :'T3-0.20','T4' :'T4-0.05'}
    for name in Raw["SensorName"] :
        ss = name
        
    label_name = ss+str(dict_label[str(label)])
    ax1.plot(Raw['Time'], Raw[str(label)], color=coloration[random_color,:],label=label_name,linewidth = 3)
    plt.xticks(Raw.Time[::step])
    plt.title(ss,fontsize = 20)
    ax1.set_xlabel('Time')
    ax1.set_ylabel('Temp [C]')

# Mise en place d'une interpolation sur la PT° et la RMS

In [ ]:
def dict_rms_function(dict_sensor,dict_value_IR,dict_temp_sensor,pseudo_temp) :
    """
    Création d'un dictionnaire de sonde contenant toutes les données relatives aux valeurs de RMS/RMSE des PT°,
    contient aussi timestamps de chaque PT°
    """
    dict_rms = {}
    dict_initial_value = {}
    for k,sensor in enumerate(dict_sensor) :
        ls_values = [] # listes des valeurs
        ls_time   = [] # listes des temps 
        ls_var    = [] # listes des écarts-types
        rms       = [] # listes des RMS
        ls_slice_value = []
        
        for IR in dict_value_IR :
            ls_values.append(pseudo_temp[sensor][IR]["values"])
            ls_time.append(pseudo_temp[sensor][IR]["time"])
            ls_var.append(pseudo_temp[sensor][IR]["var"])
            print(ls_time)

        SLICE = dict_temp_sensor[sensor] # valeurs relative à la sonde de température

        ## valeurs de RMS
        for w in range(len(ls_time)) :
            y_actual    = SLICE.loc[SLICE["Time"]==ls_time[w]]["T4"]
            y_predicted = ls_values[w]
            rms.append(np.mean(y_actual-y_predicted))

        # Affichage des valeurs de RMSE
        for time in ls_time : # compare uniquement les pseudo-temp et les valeurs de sonde 
            ls_slice_value.append(float(SLICE["T4"].loc[SLICE['Time'] == time]))
        RMSE = mean_squared_error(ls_slice_value,ls_values,squared=False)
        
        dict_rms[sensor] = {"RMS" : rms ,
                            "Time" : ls_time ,
                            "RMSE" : RMSE, 
                            'pseudo_temp' : ls_values }
        
        print("completed :",sensor)
        
    return dict_rms 
        
    

In [ ]:
dict_rms = dict_rms_function(dict_sensor,dict_value_IR,dict_temp_sensor,pseudo_temp)

### Interpolation sur la RMS / Création de dict_sensor_calibration

In [ ]:
def interpolate_RMS(dict_sensor,dict_rms):
    """
    Fonction d'interpolation des valeurs de RMS défini comme la différence entre la PT° et la T° d'une sonde
    Renvoi un dictionnaire trié par sonde comprenant une valeur toutes les 15 minutes, (pas de DataFrame)
    """
    dict_sensor_calibration = {}
    for sensor in dict_sensor :
        
        x = [(times.minute + times.hour * 60) for times in dict_rms[sensor]["Time"]] # conversion Timestamp to minutes
        y = dict_rms[sensor]["RMS"]
        start_day = dict_rms[sensor]["Time"][0]
        end_day = dict_rms[sensor]["Time"][-1]
        
        f = interp1d(x,y,kind = "cubic")
        xnew = [(times.minute + times.hour * 60) for times in pd.date_range(start_day,end_day,freq = "15min")]
        # création d'une valeur toutes les 15 minutes
        ynew = f(xnew)
        dict_sensor_calibration[sensor] = {"Interpolate_time" : pd.date_range(start_day,end_day,freq = "15min"),
                                      "Interpolate_value" : ynew }
        
        print("completed :",sensor)
    return dict_sensor_calibration
    

In [ ]:
dict_sensor_calibration = interpolate_RMS(dict_sensor,dict_rms)

# Ajout d'une colonne label dans le DataFrame dict_sensor (dictionnaire des sondes )

In [ ]:
filetif = ['6H55','7H29','8H22','9H28','10H22','11H27','12H31','13H26',
                               '15H59','17H27']
ls_path_tif_IR,filetif = get_tif(filetif)

### Visible

In [ ]:
requested_sensor = ["S1","S2","S3-AIR","S4","S5","S6","S7","S8","S9","S10"]
cluster_tif = "./traitement_PIREN/vis_piren_phase_1_cropped_clustered_7k.tif"

# dict_sensor : contient toutes les valeurs IR pour chaque sonde

with rio.open(cluster_tif) as dataset :
    for sensor in dict_sensor : 
        ls_test = []
        for path in ls_path_tif_IR :
            
            # A modifier : on retrouve le nom de l'IR 
            IR = "IR_" + path[path.find('_',20)+1 : path.find('_',23) ]
            # list des coordonnées x et y des IRs 
            x = list(dict_sensor[sensor].loc[dict_sensor[sensor]["IR"]== IR]["x"])
            y = list(dict_sensor[sensor].loc[dict_sensor[sensor]["IR"]== IR]["y"])

            for k in range(len(x)) :

                for value in dataset.sample([(float(x[k]),float(y[k]))]) : 

                    ls_test.append(int(value))
        print(sensor)
        dict_sensor[sensor]["clustering_label_vis"] = np.array(ls_test,dtype = float)

    

### HSV

In [ ]:
requested_sensor = ["S1","S2","S3-AIR","S4","S5","S6","S7","S8","S9","S10"]
cluster_tif = "./traitement_PIREN/vis_piren_phase_1_cropped_HSV_clustered_7k.tif"

# dict_sensor : contient toutes les valeurs IR pour chaque sonde

with rio.open(cluster_tif) as dataset :
    for sensor in dict_sensor : 
        ls_test = []
        for path in ls_path_tif_IR :
            
            # A modifier : on retrouve le nom de l'IR 
            IR = "IR_" + path[path.find('_',20)+1 : path.find('_',23) ]
            # list des coordonnées x et y des IRs 
            x = list(dict_sensor[sensor].loc[dict_sensor[sensor]["IR"]== IR]["x"])
            y = list(dict_sensor[sensor].loc[dict_sensor[sensor]["IR"]== IR]["y"])

            for k in range(len(x)) :

                for value in dataset.sample([(float(x[k]),float(y[k]))]) : 

                    ls_test.append(int(value))
        print(sensor)
        dict_sensor[sensor]["clustering_label_hsv"] = np.array(ls_test,dtype = float)

    

### TEST

In [ ]:
cluster_tif = "./traitement_PIREN/vis_piren_phase_1_cropped_HSV_clustered_7k.tif"
with rio.open(cluster_tif) as dataset :
    print(dataset.res)

## Graphique : Pseudo-Temps vs Temp pour toutes les profondeurs

Représenter le niveau de linéarité entre les pseudos-temépératures (moyenne du signal IR autour d'une sonde) et les températures des sondes à toutes profondeurs.
<ul>
<li> un graphique / heure</li>
<li> une couleur par profondeur</li>
</ul>

dict_temp_sensor[sensor] ##Contient les données brutes de température pr chaque capteur 
pseudo_temp[sensor][IR]['values'] ## Contient les valeurs de pseudo-température par sensor/ par IR
pseudo_temp[sensor][IR]['time'] ## Contient l'heure au format Timestamp d'une IR

In [ ]:
dict_PT_all_temp = {}
for IR in dict_IR :
    T1 = []
    T2 = []
    T3 = []
    T4 = []
    Sensor_list = {}
    IR_temp_value = []
    for sensor in dict_temp_sensor :
        if sensor != 'S3-AIR' :
            IR_time = pseudo_temp[sensor][IR]['time'] # Heure de vol
            IR_temp_value.append(pseudo_temp[sensor][IR]['values']) # liste des pseudo-températures
            Sensor_list.update({sensor : pseudo_temp[sensor][IR]['values']})
            
            # Liste des température , T1 : 0.5 / T2 : 0.35 / T3 : 0.20 / T4 : 0.05
            T1.append(dict_temp_sensor[sensor].loc[dict_temp_sensor[sensor]["Time"] == IR_time]["T1"].to_list()[0])
            T2.append(dict_temp_sensor[sensor].loc[dict_temp_sensor[sensor]["Time"] == IR_time]["T2"].to_list()[0])
            T3.append(dict_temp_sensor[sensor].loc[dict_temp_sensor[sensor]["Time"] == IR_time]["T3"].to_list()[0])
            T4.append(dict_temp_sensor[sensor].loc[dict_temp_sensor[sensor]["Time"] == IR_time]["T4"].to_list()[0])
    
    
    dict_PT_all_temp[IR] = {"T1" : T1,
                            "T2" : T2,
                            "T3" : T3,
                            "T4" : T4,
                            "IR_time" : IR_time,
                            "IR_temp_value" : IR_temp_value,
                            "Sensor_list" : Sensor_list}
    

In [ ]:
dict_PT_all_temp.keys()

## Save

In [ ]:
#np.save("dict_PT_all_temp_labelized_cluster.npy",dict_PT_all_temp)

## PT vs T pour chaque thermomosaïque (Rapport)

In [ ]:
# Liste popur chacun des capteurs
Temp_T1= []
Temp_T2= []
Temp_T3= []
Temp_T4= []
Pseudo_Temperature_list = []
for flight in dict_PT_all_temp :
    IR = dict_PT_all_temp[flight]
    Pseudo_Temperature = IR['IR_temp_value']
    for ii,TEMP in enumerate(["T1","T2","T3","T4"]) :
        Temp = IR[TEMP]
    for TEMP in ["T1"] :
        Temp_T1.append(IR[TEMP])
    for TEMP in ["T2"] :
        Temp_T2.append(IR[TEMP])
    for TEMP in ["T3"] :
        Temp_T3.append(IR[TEMP])
    for TEMP in ["T4"] :
        Temp_T4.append(IR[TEMP])
    Pseudo_Temperature_list.append(Pseudo_Temperature)
    

In [ ]:
# Paramètres d'affichage
requested_IR = ['IR_6H55']
color = ["orangered","dodgerblue","goldenrod","seagreen"]
LABEL = ['T1-0.50','T2-0.35','T3-0.20','T4-0.05']
list_markersize = [12,8,8,8]



for flight in dict_PT_all_temp :
    fig,ax = plt.subplots(1,figsize=(7,7))
    IR = dict_PT_all_temp[flight]
    Pseudo_Temperature = IR['IR_temp_value']
    for ii,TEMP in enumerate(["T1","T2","T3","T4"]) :
        Temp = IR[TEMP] # list des températures par capteur
        corr = np.corrcoef(np.array([Pseudo_Temperature]), np.array([Temp]))[0, 1] # coeff de pearson
        
        ax.plot(Pseudo_Temperature, Temp, color[ii],
                marker = 'o',linestyle= 'none',markersize = list_markersize[ii],
                label = LABEL[ii]+", corr : " + "{:.2f}".format(corr)+ ", $\sigma^{2}$ :" + "{:.2f}".format(np.asarray(Temp).var()))

    ax.set_xlabel("Pseudo-Température", size = 15)
    ax.set_xlim([-3,25])
    ax.set_ylabel("Température des sondes T(°C)", size = 13)
    ax.set_ylim([-3,25])
    ax.set_title(flight)
    ax.grid()
    ax.legend(fontsize = 14 )

    SAVE = True
    if SAVE : 
        filepath = "C:/Users/Alexandre/Desktop/Cours/Stage/Rapport/Lien PT_T/" + str(flight)+'.png'
        plt.savefig(filepath, bbox_inches="tight")
        filepath = "C:/Users/Alexandre/Desktop/Cours/Stage/Modification/Lien PT_T/" + str(flight)+'.png'
        plt.savefig(filepath, bbox_inches="tight")

    


## Distribution des données : Chaque capteur pour une thermomosaïque donnée

In [ ]:
## Création d'un dictionnaire contenant les valeurs de distribution pr toutes les IRs
requested_IR = ['IR_6H55']
color = ["orangered","dodgerblue","goldenrod","seagreen"]
LABEL = ['T1-0.50','T2-0.35','T3-0.20','T4-0.05']
dict_corr = {} # Contient toutes les DataFrames
for flight in dict_PT_all_temp :
    IR = dict_PT_all_temp[flight]
    Pseudo_Temperature = IR['IR_temp_value']
    dict_IR_corr = {} # contient les DataFrames par IR 
    for ii,TEMP in enumerate(["T1","T2","T3","T4"]) :
        Temp = IR[TEMP]
        Primary = pd.DataFrame()
        ## Calcul du normal score -> Primary/Secondary
        Primary["Temp"] = Temp
        Primary["Primary"] = (Primary["Temp"] - Primary["Temp"].mean())/ Primary["Temp"].std()

        Primary["pseudo_temp"] = Pseudo_Temperature
        Primary["Secondary"] = (Primary["pseudo_temp"] - Primary["pseudo_temp"].mean())/ Primary["pseudo_temp"].std()
        dict_IR_corr[TEMP] = Primary
        
    dict_corr[flight] = dict_IR_corr
        
        
        
        

       

In [ ]:
requested_IR = dict_corr['IR_10H22']
requested_filename = 'IR_10H22'

for TEMP in requested_IR :
    Primary = requested_IR[TEMP]
    # Set up the axes with gridspec
    corr = np.corrcoef(Primary["Primary"], Primary["Secondary"])[0, 1]
    vlim = (-3, 3)
    fig = plt.figure(figsize=(6, 6))
    grid = plt.GridSpec(4, 4, hspace=0.5, wspace=0.5)
    main_ax = fig.add_subplot(grid[:-1, 1:])
    y_hist = fig.add_subplot(grid[:-1, 0], xticklabels=[], sharey=main_ax)
    x_hist = fig.add_subplot(grid[-1, 1:], yticklabels=[], sharex=main_ax)

    # scatter points on the main axes
    main_ax.plot(Primary["Primary"], Primary["Secondary"], "ok", markersize=3)
    main_ax.set_xlim(vlim)
    main_ax.set_ylim(vlim)
    main_ax.set_title(requested_filename +' : '+ TEMP , size=15)
    main_ax.text(-2.5, 2.2,
        "Corr = {0:.3f}".format(
            np.corrcoef(Primary["Primary"], Primary["Secondary"])[0, 1]
        ),
        size=15
    )

    # histogram on the attached axes
    x_hist.hist(
        Primary["Primary"],
        40,
        histtype="stepfilled",
        label="Primary",
        orientation="vertical",
        color="gray",
        range=vlim,
    )
    x_hist.set_xlabel("Primary Histogram", size=20)
    x_hist.invert_yaxis()


    y_hist.hist(
        Primary["Secondary"],
        40,
        histtype="stepfilled",
        orientation="horizontal",
        color="gray",
        range=vlim,
    )
    y_hist.set_ylabel("Secondary Histogram", size=20)
    y_hist.invert_xaxis()
    
    SAVE = True
    if SAVE : 
        filepath = "C:/Users/Alexandre/Desktop/Cours/Stage/CoCalc/DTP/linear_all_IR/distr/" + requested_filename +'_'+TEMP+'.png'
        plt.savefig(filepath, bbox_inches="tight")

## PT vs T sol pour tous les vol par sonde

#### dict_PT_single_sensor : contient les valeurs T1->T4 pour une unique sonde

In [ ]:
dict_PT_single_sensor = {}
for IR in dict_IR :
    T1 = {}
    T2 = {}
    T3 = {}
    T4 = {}
    Sensor_list = {}
    IR_temp_value = []
    for sensor in dict_temp_sensor :
        if sensor != 'S3-AIR' :
            IR_time = pseudo_temp[sensor][IR]['time'] # Heure de vol
            IR_temp_value.append(pseudo_temp[sensor][IR]['values']) # liste des pseudo-températures
            Sensor_list.update({sensor : pseudo_temp[sensor][IR]['values']})
            
            # Liste des température , T1 : 0.5 / T2 : 0.35 / T3 : 0.20 / T4 : 0.05
            T1.update({sensor : dict_temp_sensor[sensor].loc[dict_temp_sensor[sensor]["Time"] == IR_time]["T1"].to_list()[0]})
            T2.update({sensor : dict_temp_sensor[sensor].loc[dict_temp_sensor[sensor]["Time"] == IR_time]["T2"].to_list()[0]})
            T3.update({sensor : dict_temp_sensor[sensor].loc[dict_temp_sensor[sensor]["Time"] == IR_time]["T3"].to_list()[0]})
            T4.update({sensor : dict_temp_sensor[sensor].loc[dict_temp_sensor[sensor]["Time"] == IR_time]["T4"].to_list()[0]})
    
    
    dict_PT_single_sensor[IR] = {"T1" : T1,
                                 "T2" : T2,
                                 "T3" : T3,
                                 "T4" : T4,
                                 "IR_time" : IR_time,
                                 "IR_temp_value" : IR_temp_value,
                                 "Sensor_list" : Sensor_list}
    

In [ ]:
# Paramètres d'affichage

requested_sensor = dict_PT_single_sensor["IR_6H55"]["Sensor_list"] # affiche toutes les sondes
color = ["orangered","dodgerblue","goldenrod","seagreen"]
LABEL = ['T1-0.50','T2-0.35','T3-0.20','T4-0.05']
list_markersize = [10,6,8,8]

for sensor in requested_sensor :
    fig,ax = plt.subplots(1,figsize=(7,7))
    Pseudo_Temperature = []
    count = 0
    for flight in dict_PT_single_sensor :
        IR = dict_PT_single_sensor[flight]
        Pseudo_Temperature = IR['Sensor_list'][sensor]
        for ii,TEMP in enumerate(["T1","T2","T3","T4"]) :
            Temp = IR[TEMP][sensor]
            
            if count <= 3 :
                #corr = np.corrcoef(np.array([Pseudo_Temperature]), np.array([Temp]))[0, 1]
                ax.plot(Pseudo_Temperature, Temp, color[ii],
                        marker = 'o',linestyle= 'none',markersize = list_markersize[ii],
                        label = LABEL[ii])
            else :
                ax.plot(Pseudo_Temperature, Temp, color[ii],
                        marker = 'o',linestyle= 'none',markersize = list_markersize[ii])
            
            count +=1
            #print(TEMP,sensor,flight,Temp,  "|")
            
    x = np.arange(-5,28)
    y = np.arange(-5,28)
    ax.plot(x,y,"grey")
    ax.legend(loc = "best")
    ax.set_xlabel("Pseudo-Température", size = 15)
    ax.set_xlim([-5,25])
    ax.set_ylabel("Température des sondes T(°C)", size = 13)
    ax.set_ylim([-5,25])
    ax.set_title(sensor)
    ax.grid()

    SAVE = True
    if SAVE : 
        print(sensor)
        filepath = "C:/Users/Alexandre/Desktop/Cours/Stage/Modification/PT_T en fonction des sondes/" + str(sensor)+'.png'
        plt.savefig(filepath, bbox_inches="tight")

    


## Re-sampling du cluster

### Fichier .tif contenant les clusters 

` fileout_cluster_hsv = "./traitement_PIREN/vis_piren_phase_1_cropped_HSV_clustered_7k.tif"
print("file cluster :",fileout_cluster_hsv)
dict_windows_cluster_hsv, dict_windows_param_cluster_hsv = reading_cluster(4,fileout_cluster_hsv)`

### Fichier original 

`filename_hsv = "./traitement_PIREN/vis_piren_phase_1_cropped_HSV.tif"
print("file 3band :",filename_hsv)
dict_windows_hsv, dict_windows_param_hsv = reading_3band(4,filename_hsv,False)`

In [ ]:
fileout_cluster_hsv = "./traitement_PIREN/vis_piren_phase_1_cropped_HSV_clustered_7k.tif"
print("file cluster :",fileout_cluster_hsv)
fileout_IR = "./traitement_PIREN/001_6H55_ortho_UTM31N_IR.tif"
print("fileout IR :",fileout_IR)

##### TODO :
1. Resampling du cluster : JE pars du principr qu'on utilise seulement celle sur HSV ?? malgré que VIS isole bien les plaques (DONE)___
        1. Coeff de resampling différent d'une IR à l'autre (DONE)___
2. Transformation en DataFrame (DONE)___
3. Ajout d'un label Cluster à l'ensemble d'une IR (?? vraiment nécessaire ??)
4. Isolation des points autour d'une sonde (deux rayons blablabla)
5. On choisi le groupe qui possède la plus grosse proportion dans les points (~1800-2000pts par Pseudo-Temp)

In [ ]:
def Re_sampling_function(filetif_dataset,ls_filetif_res,filetif) :
    """
    Re-sampling avec un fichier temporaire "sampled.tif" qui permet de faire correspondre deux .tif avec comme
    base la résolution (x,y) du pixel
    filetif_dataset : chemin d'accès du .tif à re-sampler
    filetif_res : chemin d'accès du '.tif' à la résolution (x,y) cible
    
    """
    ls_fileout = []
    for kk,filetif_res in enumerate(ls_filetif_res) : 
        ## On récupère la résolution cible
        with rio.open(filetif_res) as dataset_res :
                #print("dataset_res : ",dataset_res.res)
                res_x = dataset_res.res[0]
                res_y = dataset_res.res[1]

        ## On récupère le dataset cible
        with rio.open(filetif_dataset) as dataset :
                #print("dataset : ",dataset.res)
                res_dataset_x = dataset.res[0]
                res_dataset_y = dataset.res[1]

                # Facteurs liés au resampling 
                upscale_factor_height = res_dataset_y / res_y
                upscale_factor_width = res_dataset_x / res_x

                # resample data to target shape
                data = dataset.read(
                    out_shape=(
                        dataset.count,
                        int(dataset.height * upscale_factor_height),
                        int(dataset.width * upscale_factor_width)),
                    resampling=Resampling.bilinear
                )

                # scale image transform
                transform = dataset.transform * dataset.transform.scale(
                    (dataset.width / data.shape[-1]),
                    (dataset.height / data.shape[-2])
                )

                # Save the new profile & .tif
                profile = {
                    "driver": "GTiff",
                    "count": 1,
                    "height": int(dataset.height * upscale_factor_height),
                    "width": int(dataset.width * upscale_factor_width),
                    "dtype": "uint8",
                    "transform": transform,
                    "bounds": dataset.bounds,
                    "crs": dataset.crs,
                    "res": (transform[0],transform[4])
                }
                # Filepath 
                fileout = "./traitement_PIREN/sampled_"+str(filetif[kk])+".tif"
                print("new shape :", data.shape[1:3])
                with rio.open(fileout,"w",**profile) as dataset_sample :
                    dataset_sample.write(data)
                    print("Completed, new file can be found :",fileout)
                    ls_fileout.append(fileout)
                
    return ls_fileout
            

In [ ]:
def get_value_pseudo_temp_clust(ls_path_tif,ls_filepath_cluster,dict_IR,r1 = 0.7 , r2 = 1.5) :
    """
    Création d'un dictionnaire classé par IR contenant toutes les données comprise entre un rayon r1 et r2, 
    """
    
    dict_value_IR = {}

    for i,IR in enumerate(dict_IR) :
        dict_value_sensor = {}
        with rio.open(ls_path_tif[i]) as dataset : # ouverture du fichier obligatoire pour recup la valeur
            for sensor in dict_IR[IR]["dict_windows"] :

                res = dict_IR[IR]["dict_windows_param"][sensor]["resolution"] # resolution [x,y]
                sensor_dataframe = dict_IR[IR]["dict_windows"][sensor]
                mid_indice = (dict_IR[IR]["dict_windows_param"][sensor]['win'].width) # les indices commencent à 0 !

                mid_x = sensor_dataframe.iloc[len(sensor_dataframe)//2]["x"] 
                mid_y = sensor_dataframe.iloc[(mid_indice//2)-1]["y"]# la valeur du milieu est //2 

                #list des valeurs autour de [0, 2pi] , mouvement anti-horaire
                theta = np.linspace(0,np.pi*2,round(np.pi*2/res[0]))
                ls_x = np.empty([1]) ; ls_y = np.empty([1])

                #recherche de points entre deux rayon r1 et r2
                
                for THETA in theta :
                    #
                    x1 = mid_x + r1*np.cos(THETA)
                    y1 = mid_y + r1*np.sin(THETA)

                    x2 = mid_x + r2*np.cos(THETA)
                    y2 = mid_y + r2*np.sin(THETA)

                    ls_x = np.append(ls_x,np.linspace(x1,x2,round((r2-r1)/res[0])+1)) 
                    ls_y = np.append(ls_y,np.linspace(y1,y2,round((r2-r1)/res[1])+1))

                ls_x = np.delete(ls_x,0) ; ls_y = np.delete(ls_y,0)


                with rio.open(ls_filepath_cluster[i]) as dataset_cluster : # ouverture du fichier obligatoire pour recup la valeur

                    dict_IR[IR]["dict_windows_param"][sensor]['win'].width # résolution -> nb de pixel == nb d'indice 
                    ls_cluster = []
                    ls_value   = []
                    for k in range(len(ls_x)) :

                        for value in dataset_cluster.sample([(float(ls_x[k]),float(ls_y[k]))]) : 
                            ls_cluster.append(float(value))
                        
                        for value in dataset.sample([(float(ls_x[k]),float(ls_y[k]))]) : 
                            ls_value.append(float(value))

                    dict_value_sensor[sensor] = pd.DataFrame(
                        np.array([ls_x,ls_y,ls_cluster,ls_value],
                                 dtype = float).T,columns = ["x","y","cluster","value"])
        print(IR)
        dict_value_IR[IR] = dict_value_sensor
        print("nombre de coordonnées utilisée pour la moyenne :",dict_value_IR[IR][sensor].shape[0])
        
    return dict_value_IR

In [ ]:
def evaluate_pseudo_temp_clust(dict_value_IR,ls_fileout) :
    """
    Fonction qui permet d'établir les valeurs de Pseudo-température à l'aide des clusters, 
    La pseudo-temp correspond à la valeur moyenne du cluster comportant le plus de pixel autout d'une sonde
    """
    
    requested_sonde = ['S1', 'S10', 'S2', 'S3-AIR', 'S4', 'S5', 'S6', 'S7', 'S8', 'S9']
    pseudo_temp = {}
    
    for sonde in requested_sonde :
        pseudo_temp_IR = {}
        for IR in dict_value_IR :
            print(IR)
            df = dict_value_IR[IR][sonde]
            last_count = 0
            clust_group = 0
            for ii in np.arange(df["cluster"].min(),df["cluster"].max()+1) : 
                count = len(df["cluster"].loc[df["cluster"]== int(ii)].values)
                if count > last_count :
                    count = last_count
                    clust_group = ii
            
            
            var = df.loc[df["cluster"]== int(ii)]["value"].std()
            mean = df.loc[df["cluster"]== int(ii)]["value"].mean()
            print(sonde,IR,"mean :",mean)
            pseudo_temp_IR[IR] = {"values" : mean,
                                  "var" : var,
                                  "time" : np.NAN }
            #print("var :",pseudo_temp_IR[IR]['var'])

            pseudo_temp_IR[IR]['time'] = pd.Timestamp(str('2019-04-14' +' '
                                                          +IR[IR.find('_')+1:IR.find(('H'))] + ':' + 
                                                          IR[IR.find('H')+1::] )).round('30min')
            
        
        pseudo_temp[sonde] = pseudo_temp_IR
        
    for fileout in ls_fileout :
        print("suppress IR :",fileout)
        rio.shutil.delete(fileout)
        
    return pseudo_temp

In [ ]:
filetif = ['6H55','7H29','8H22','9H28','10H22','11H27','12H31','13H26',
                               '15H59','17H27']
#filetif = ['6H55']
ls_path_tif,filetif = get_tif(filetif)
Dimension = 4 # en m

## HSV
ls_fileout_hsv = Re_sampling_function(fileout_cluster_hsv,ls_path_tif,filetif)
#filetif_sampled = list("sampled_"+file for file in filetif)

# Dictionnaire fenêtrage autour des sondes 
dict_IR_hsv = reading_1band(Dimension,filetif,ls_fileout_hsv,LABEL = "cluster")
# Dictionnaire contenant les valeurs comprise entre les deux rayons de recherche
dict_value_hsv = get_value_pseudo_temp_clust(ls_path_tif,ls_fileout_hsv,dict_IR_hsv,r1 = 0.7 , r2 = 1.5)
# Dictionnaire contenant les valeurs pseudo-température
pseudo_temp = evaluate_pseudo_temp_clust(dict_value_hsv,ls_fileout_hsv)

In [ ]:
pseudo_temp

In [ ]:
for sonde in ["S1"] :
        pseudo_temp_IR = {}
        for IR in ["IR_6H55"] :
            print(IR)
            df = dict_value_hsv[IR][sonde]
            last_count = 0
            clust_group = 0
            for ii in np.arange(df["cluster"].min(),df["cluster"].max()+1) : 
                count = len(df["cluster"].loc[df["cluster"]== int(ii)].values)
                if count > last_count :
                    count = last_count
                    clust_group = ii

In [ ]:
df

# PARTIE GRAPHIQUE

# Boxplot : normalization différente pour chaque IR

## Plot de plusieurs sondes sans patch

In [ ]:
import seaborn as sns
requested_sensor = ["S1","S2","S3-AIR","S4","S5"]
fig,ax = plt.subplots(len(requested_sensor),1,figsize=(20,30))
for k,sensor in enumerate(requested_sensor) :
    with sns.color_palette("husl") as cmap :
        data = dict_sensor[sensor]

        ax[k].set_ylim(0, 1)
        sns.boxplot(x='IR',y="band 1",
                   data = data,palette = cmap , ax = ax[k])
        sns.set_theme(style="ticks",font_scale = 2)

        sns.despine(offset=10, trim=True)
        
        ax[k].set_title(sensor)
plt.subplots_adjust(left=None, bottom=None, right=None, top=None ,wspace=None, hspace=0.8)
        

## Boxplot d'une sonde accompagné des patchs IR et IR normalisée

In [ ]:
import seaborn as sns
from matplotlib.gridspec import GridSpec
requested_sensor = ["S3-AIR"]

fig = plt.figure(figsize=(30,20))

gs = GridSpec(3, 10, figure=fig)
gs.update(wspace = 0.4, hspace=0.0) # set the spacing between axes.

for sensor in requested_sensor :
    with sns.color_palette("husl") as cmap :
        data = dict_sensor[sensor]
        
        ax = fig.add_subplot(gs[0, :])
        
        ax.set_ylim(0,1)
        sns.boxplot(x='IR',y="band 1",
                   data = data,palette = cmap,ax = ax )
        sns.set_theme(style="ticks",font_scale = 2)
        sns.despine(offset=10, trim=True)
        LONGUEUR = dict_IR["IR_6H55"]["dict_windows_param"]["S10"]["LONGUEUR"]
        
        ax.set_ylabel("Normalisation commune",fontsize = 20)
        plt.title(sensor + ' '+str(LONGUEUR)+ 'm patch')
        
        for k,IR in enumerate(dict_IR) :
            ax2 = fig.add_subplot(gs[1, k])
            band_min = dict_IR[IR]["dict_windows_param"][requested_sensor[0]]["all_band"].min()
            band_max = dict_IR[IR]["dict_windows_param"][requested_sensor[0]]["all_band"].max()
            
            window_IR_Normalize = ((dict_IR[IR]["dict_windows_param"][requested_sensor[0]]["all_band"]-band_min)/(band_max-band_min)) 
            window_transform_IR = dict_IR[IR]["dict_windows_param"][requested_sensor[0]]["win_transform"]
            extent = plotting_extent(window_IR_Normalize,window_transform_IR)
            sub_fig = ax2.imshow(window_IR_Normalize,extent = extent,cmap ='viridis',vmin = 0,vmax = 1)
            ax2.set_title(IR)
            ax2.set_xticks([])
            ax2.set_yticks([])
            fig.colorbar(sub_fig,ax=ax2,fraction=0.046, pad=0.04)
            
            ax3 = fig.add_subplot(gs[2, k])
            window_IR = dict_IR[IR]["dict_windows_param"][requested_sensor[0]]["all_band"]
            window_transform_IR = dict_IR[IR]["dict_windows_param"][requested_sensor[0]]["win_transform"]
            extent = plotting_extent(window_IR,window_transform_IR)
            sub_fig2 = ax3.imshow(window_IR,extent = extent,cmap ='viridis')
            fig.colorbar(sub_fig2,ax=ax3,fraction=0.046, pad=0.04)
            
            
            
            

# Boxplot : normalization commune des IR

## Plot de plusieurs sondes sans patch

In [ ]:
import seaborn as sns
requested_sensor = ["S6","S7","S8","S9","S10"]
fig,ax = plt.subplots(len(requested_sensor),1,figsize=(20,30))
for k,sensor in enumerate(requested_sensor) :
    with sns.color_palette("husl") as cmap :
        
        data = dict_sensor[sensor]

        ax[k].set_ylim(0, 1)
        sns.boxplot(x='IR',y="band N",
                   data = data,palette = cmap,ax = ax[k] )
        sns.set_theme(style="ticks",font_scale = 2)
        
        ax[k].set_ylabel("Normalisation commune",fontsize = 20)
        ax[k].set_title(sensor)
        
        sns.despine(offset=10, trim=True)
        
plt.subplots_adjust(left=None, bottom=None, right=None, top=None ,wspace=None, hspace=0.8)
        

## Boxplot d'une sonde accompagné des patchs IR et IR normalisée

In [ ]:
import seaborn as sns
from matplotlib.gridspec import GridSpec
requested_sensor = ["S10"]

fig = plt.figure(figsize=(30,20))

gs = GridSpec(3, 10, figure=fig)
gs.update(wspace = 0.4, hspace=0.0) # set the spacing between axes.

            
for sensor in requested_sensor :
    # Valeurs requises pour la normalisation 
    BAND_MIN_N = 0
    BAND_MAX_N = 0
    
    for target in dict_IR :
        min_= dict_IR[target]["dict_windows"][sensor]["band 1"].min()
        max_= dict_IR[target]["dict_windows"][sensor]["band 1"].max()
        if min_ < BAND_MIN_N :
            BAND_MIN_N = min_


        if max_ > BAND_MAX_N :
            BAND_MAX_N = max_
    
    print("BAND_MIN_N",BAND_MIN_N) # valeur minimale retenue pour la normalisation
    print("BAND_MAX_N",BAND_MAX_N) # valeur maximale retenue pour la normalisation
    
    with sns.color_palette("husl") as cmap :
        data = dict_sensor[sensor]
        
        ax = fig.add_subplot(gs[0, :])
        
        ax.set_ylim(0,1)
        sns.boxplot(x='IR',y="band N",
                   data = data,palette = cmap,ax = ax )
        sns.set_theme(style="ticks",font_scale = 2)
        sns.despine(offset=10, trim=True)
        LONGUEUR = dict_IR["IR_6H55"]["dict_windows_param"]["S10"]["LONGUEUR"]
        
        ax.set_ylabel("Normalisation commune",fontsize = 20)
        plt.title(sensor + ' '+str(LONGUEUR)+ 'm patch')
        
        # Plot des IRs Normalisées et Non Normalisées
        for k,IR in enumerate(dict_IR) :
            ax2 = fig.add_subplot(gs[1, k])
            window_IR_Normalize = ((dict_IR[IR]["dict_windows_param"][requested_sensor[0]]["all_band"]-BAND_MIN_N)/(BAND_MAX_N-BAND_MIN_N)) 
            window_transform_IR = dict_IR[IR]["dict_windows_param"][requested_sensor[0]]["win_transform"]
            extent = plotting_extent(window_IR_Normalize,window_transform_IR)
            sub_fig = ax2.imshow(window_IR_Normalize,extent = extent,cmap ='viridis',vmin = 0,vmax = 1)
            ax2.set_title(IR)
            ax2.set_xticks([ ])
            ax2.set_yticks([ ])
            fig.colorbar(sub_fig,ax=ax2,fraction=0.046, pad=0.04)
            
            ax3 = fig.add_subplot(gs[2, k])
            window_IR = dict_IR[IR]["dict_windows_param"][requested_sensor[0]]["all_band"]
            window_transform_IR = dict_IR[IR]["dict_windows_param"][requested_sensor[0]]["win_transform"]
            extent = plotting_extent(window_IR,window_transform_IR)
            sub_fig2 = ax3.imshow(window_IR,extent = extent,cmap ='viridis')
            fig.colorbar(sub_fig2,ax=ax3,fraction=0.046, pad=0.04)

fileout =  "C:/Users/Alexandre/Desktop/Cours/Stage/Temp_vs_IR/2m/Normalisation commune/"+ str(sensor)+".png"
fig.savefig(fileout,bbox_inches = 'tight')
            
            
            

##### Liste de keys 

In [ ]:
dict_IR.keys()

In [ ]:
dict_IR['IR_6H55'].keys()

In [ ]:
dict_IR['IR_6H55']["dict_windows_param"]["S10"].keys()


# Plot d'une sonde + Pseudo Temp° + IR + Clustering

In [ ]:
requested_sensor = ["S1","S2","S3-AIR","S4","S5","S6","S7","S8","S9","S10"]

In [ ]:
from matplotlib.gridspec import GridSpec
from sklearn.metrics import mean_squared_error
requested_sensor = ["S1","S2","S3-AIR","S4","S5","S6","S7","S8","S9","S10"]
#n = dict_windows_param_cluster_hsv[sensor]["all_band"].max()
n = 6
for sensor in requested_sensor :
    
    # Propriétés graphiques
    fig = plt.figure(figsize=(25,15))
    SMALL_SIZE = 10
    MEDIUM_SIZE = 12
    BIGGER_SIZE = 15
    plt.rc('font', size=BIGGER_SIZE)          # controls default text sizes
    plt.rc('axes', titlesize=BIGGER_SIZE)     # fontsize of the axes title !!! 
    plt.rc('axes', labelsize=BIGGER_SIZE)    # fontsize of the x and y labels
    plt.rc('xtick', labelsize=BIGGER_SIZE)    # fontsize of the tick labels
    plt.rc('ytick', labelsize=BIGGER_SIZE)    # fontsize of the tick labels
    plt.rc('legend', fontsize=BIGGER_SIZE)    # legend fontsize
    plt.rc('figure', titlesize=BIGGER_SIZE)  # fontsize of the figure title
    
    ##Création des figures
    gs = GridSpec(3, 10, figure=fig)
    gs.update(wspace = 0.5, hspace=0.0) # set the spacing between axes.
    ax = fig.add_subplot(gs[0, :])
    
    ls_values = [] # listes des valeurs
    ls_time   = [] # listes des temps 
    ls_var    = [] # listes des écarts-types
    rms       = [] # listes des RMS
    
    print(sensor)
    for IR in dict_value_IR :
        ls_values.append(pseudo_temp[sensor][IR]["values"])
        ls_time.append(pseudo_temp[sensor][IR]["time"])
        ls_var.append(pseudo_temp[sensor][IR]["var"])
        

    
    SLICE = dict_temp_sensor[sensor]
    ## Plot de la température des sondes 
    plottingtemp_single_label_IR_a(SLICE,fig,ax,'T4',200,6)
    ax.set_ylim(-5,30)
    ax3 = ax.twinx() # instantiate a second axes that shares the same x-axis
    color = "tab:red"
    ax.set_ylim(-5,30)
    ax.grid()

    ## Plot de la pseudo Température 
    ax3.plot(ls_time,ls_values,color = color)
    ax3.plot(ls_time,ls_values,'ro',color = color)
    ax3.tick_params(axis = 'y', labelcolor = color)
    ax3.set_ylabel("Pseudo Température", color = color, fontsize = 20)
    ax3.set_ylim(-5,30)
    
    # [ val - sigma , val + sigma]
    
    ls_sigma = [np.array(ls_values)-np.array(ls_var),np.array(ls_values)+np.array(ls_var)]
    
    ## Affichage des valeurs de RMS
    for w in range(len(ls_time)) :
        y_actual    = SLICE.loc[SLICE["Time"]==ls_time[w]]["T4"] 
        y_predicted = ls_values[w]
        rms.append(np.absolute(np.mean(y_actual-y_predicted)))
        ax3.text(ls_time[w],ls_values[w]+0.5,str(rms[w])[0:5],fontsize = 15) # valeurs RMS
  
    ## Affichage de la valeur de RMSE (une valeur)
    ls_slice_value = []
    for time in ls_time : # compare uniquement les pseudo-temp et les valeurs de sonde 
        ls_slice_value.append(float(SLICE["T4"].loc[SLICE['Time'] == time]))
    
    RMSE = mean_squared_error(ls_slice_value,ls_values,squared=False)
    ax3.text(SLICE["Time"].iloc[-8],28,'RMSE = ' + str(RMSE)[0:5],fontsize = 15) # valeurs RMSE
    
    # Interval de confiance +/- sigma 
    ax3.plot(ls_time,ls_sigma[0],'m',linestyle='--',label = '- sigma') # value - sigma
    ax3.plot(ls_time,ls_sigma[1],'m',linestyle='--',label = '+ sigma') # value + sigma
    ax3.legend(loc='upper right', bbox_to_anchor=(1, 0.80))
    
    ## plot des patchs IRs
    for k,IR in enumerate(dict_IR) :
        ax2 = fig.add_subplot(gs[1, k])
        
        window_IR = dict_IR[IR]["dict_windows_param"][sensor]["all_band"]
        window_transform_IR = dict_IR[IR]["dict_windows_param"][sensor]["win_transform"]
        extent = plotting_extent(window_IR,window_transform_IR)
        
        sub_fig = ax2.imshow(window_IR,extent = extent,cmap ='viridis')
        ax2.set_title(IR,fontsize = 20,pad = 20)
        fig.colorbar(sub_fig,ax=ax2,fraction=0.046, pad=0.04)
        
        # Plot des targets
        TARGET = sensor_coord.loc[sensor_coord["SensorName"]==sensor]
        c_1 = dict_value_IR[IR][sensor]["x"].iloc[0]
        c_2 = dict_value_IR[IR][sensor]["x"].iloc[-1]
        X = np.array(TARGET["x"],dtype = float)
        Y = np.array(TARGET["y"],dtype=float)
        circle1 = plt.Circle((X,Y),(c_1-X),color='b', fill=False)
        circle2 = plt.Circle((X,Y),(c_2-X),color='r', fill=False)
        ax2.add_patch(circle1)
        ax2.add_patch(circle2)
      
    ## plot des labels de clustering 
    #visible
    ax4 = fig.add_subplot(gs[2, 0:2])
    #n = dict_windows_param_cluster[sensor]["all_band"].max()
    ole=cm.get_cmap('jet', n)
    
    window_cluster = dict_windows_param_cluster[sensor]["all_band"]
    window_cluster_transform = dict_windows_param_cluster[sensor]["win_transform"]
    extent = plotting_extent(window_cluster,window_cluster_transform)
    sub_fig = ax4.imshow(window_cluster,extent = extent,cmap = ole,vmin = 0,vmax = n)
    ax4.set_title(sensor + " cluster sur la bande visible",fontsize = 20,pad = 20)
    fig.colorbar(sub_fig,ax=ax4,fraction=0.046, pad=0.04)
    
    #hsv
    ax5 = fig.add_subplot(gs[2, 7:-1])
    window_hsv = dict_windows_param_cluster_hsv[sensor]["all_band"]
    window_transform_hsv = dict_windows_param_cluster_hsv[sensor]["win_transform"]
    extent = plotting_extent(window_hsv,window_transform_hsv)
    sub_fig = ax5.imshow(window_hsv,extent = extent,cmap = ole,vmin = 0,vmax = n)
    ax5.set_title(sensor + " cluster sur la bande HSV",fontsize = 20,pad = 20)
    fig.colorbar(sub_fig,ax=ax5,fraction=0.046, pad=0.04)
    
    ## plot du patch visible
    ax6 = fig.add_subplot(gs[2, 3:6])
    window_vis = dict_windows_param[sensor]["all_band"]
    window_transform_vis = dict_windows_param[sensor]["win_transform"]
    show(window_vis,transform = window_transform_vis,ax = ax6)
    ax6.set_title(' ',fontsize = 20,pad = 20)
    
    ## paramètres d'affichage
    ax.set_title(sensor +" et valeurs de RMS pour chaque pseudo température",fontsize = 20)
    ax.set_xticklabels([str(time.hour)+':'+str(time.minute) for time in SLICE['Time']][::8])
    ax.set_xlabel('Time (heures:minutes)' + ' 2019-04-19' )
    fileout =  "C:/Users/Alexandre/Desktop/Cours/Stage/Temp_vs_IR/2m/"+ str(sensor)+".png"
    fig.savefig(fileout)

## Courbe de différence pour toutes les sondes ( Temp° sonde - Pseudo_Temp° )

### Pour toutes les courbes 

In [ ]:
from matplotlib.gridspec import GridSpec
from sklearn.metrics import mean_squared_error
requested_sensor = ["S1","S2","S3-AIR","S4","S5","S6","S7","S8","S9","S10"]

coloration=plt.cm.tab10(np.linspace(0,1,10))
fig,ax = plt.subplots(1,figsize=(30,10))



for k,sensor in enumerate(requested_sensor) :
    
    # Propriétés graphiques
    
    SMALL_SIZE = 15
    MEDIUM_SIZE = 15
    BIGGER_SIZE = 30
    plt.rc('font', size=BIGGER_SIZE)          # controls default text sizes
    plt.rc('axes', titlesize=BIGGER_SIZE)     # fontsize of the axes title !!! 
    plt.rc('axes', labelsize=BIGGER_SIZE)    # fontsize of the x and y labels
    plt.rc('xtick', labelsize=BIGGER_SIZE)    # fontsize of the tick labels
    plt.rc('ytick', labelsize=BIGGER_SIZE)    # fontsize of the tick labels
    plt.rc('legend', fontsize=MEDIUM_SIZE)    # legend fontsize
    plt.rc('figure', titlesize=BIGGER_SIZE)  # fontsize of the figure title
    
    ls_values = [] # listes des valeurs
    ls_time   = [] # listes des temps 
    ls_var    = [] # listes des écarts-types
    rms       = [] # listes des RMS
    ls_slice_value = []
    
    
    print(sensor)
    for IR in dict_value_IR :
        ls_values.append(pseudo_temp[sensor][IR]["values"])
        ls_time.append(pseudo_temp[sensor][IR]["time"])
        ls_var.append(pseudo_temp[sensor][IR]["var"])
        
    SLICE = dict_temp_sensor[sensor]
    ## Affichage des valeurs de RMS
    for w in range(len(ls_time)) :
        y_actual    = SLICE.loc[SLICE["Time"]==ls_time[w]]["T4"]
        y_predicted = ls_values[w]
        rms.append(np.mean(y_actual-y_predicted))
    
    # Affichage des valeurs de RMSE
    for time in ls_time : # compare uniquement les pseudo-temp et les valeurs de sonde 
        ls_slice_value.append(float(SLICE["T4"].loc[SLICE['Time'] == time]))
    RMSE = mean_squared_error(ls_slice_value,ls_values,squared=False)
    dict_rms[sensor] = {"RMS" : rms , "Time" : ls_time ,"RMSE" : RMSE }
    
    ax.plot(ls_time,rms,color=coloration[k,:],label = sensor +', RMSE : '+ str(RMSE)[0:5]) 
    ax.plot(ls_time,rms,'o',color=coloration[k,:])
    
ax.set_xticks(ls_time)
ax.set_xticklabels([str(time.hour)+':'+str(time.minute) for time in ls_time])
ax.set_xlabel('Time (heures:minutes)' + ' 2019-04-19' )
ax.set_ylabel('Temp [C]')

ax.legend()
ax.grid()
ax.set_title("Courbes des différences entre les valeus de T° et des Pseudo T° ")
                   
    
        

## Méthodes d'interpolation des pseudo-températures pour sonde(s)

In [ ]:
from scipy.interpolate import interp1d

for sensor in dict_sensor :
    x = [(times.minute + times.hour * 60) for times in dict_rms[sensor]["Time"]]
    y = dict_rms[sensor]["pseudo_temp"]
    fig,ax = plt.subplots(1,figsize=(15,5))
    

    ls_kind = ["zero","slinear","quadratic","cubic"]
    color = ['gold','cyan',"chartreuse","dodgerblue"]
    for k,kind in enumerate(ls_kind) :
        start = dict_rms[sensor]["Time"][0]
        end = dict_rms[sensor]["Time"][-1]

        f = interp1d(x,y,kind =kind)
        xnew = [(times.minute + times.hour * 60) for times in pd.date_range(start,end,freq = "15min")]
        # création d'une valeur toutes les 15 minutes
        ynew = f(xnew)
        ax.plot(xnew,ynew,color = color[k],label = kind + ' ('+str(k)+' ordre)')

    ax.plot(x,y,'ro',label = "data")
    ax.set_xticklabels([str(time.hour)+':'+str(time.minute) for time in dict_rms[sensor]["Time"]])
    ax.set_xlabel('Time (heures:minutes)' + ' 2019-04-19' )
    ax.set_ylabel('Temp [C]')
    plt.legend()
    ax.set_ylim(-5,30)
    ax.set_title("Méthodes d'interpolation des pseudo-température pour la sonde "+sensor)
    ax.grid()
    fileout =  "C:/Users/Alexandre/Desktop/Cours/Stage/Temp_vs_IR/2m/debut_interpolation/"+ str(sensor)+".png"
    fig.savefig(fileout,bbox_inches = 'tight' )

In [ ]:
from scipy.interpolate import interp1d

for sensor in dict_sensor :
    x = [(times.minute + times.hour * 60) for times in dict_rms[sensor]["Time"]]
    y = dict_rms[sensor]["RMS"]
    fig,ax = plt.subplots(1,figsize=(15,5))
    

    ls_kind = ["zero","slinear","quadratic","cubic"]
    color = ['gold','cyan',"chartreuse","dodgerblue"]
    for k,kind in enumerate(ls_kind) :
        start = dict_rms[sensor]["Time"][0]
        end = dict_rms[sensor]["Time"][-1]

        f = interp1d(x,y,kind =kind)
        xnew = [(times.minute + times.hour * 60) for times in pd.date_range(start,end,freq = "15min")]
        # création d'une valeur toutes les 15 minutes
        ynew = f(xnew)
        ax.plot(xnew,ynew,color = color[k],label = kind + ' ('+str(k)+' ordre)')

    ax.plot(x,y,'ro',label = "data")
    ax.set_xticklabels([str(time.hour)+':'+str(time.minute) for time in dict_rms[sensor]["Time"]])
    ax.set_xlabel('Time (heures:minutes)' + ' 2019-04-19' )
    ax.set_ylabel('Temp [C]')
    plt.legend()
    ax.set_title("Méthodes d'interpolation des différences de pseudo-température pour la sonde "+sensor)
    ax.grid()
    fileout =  "C:/Users/Alexandre/Desktop/Cours/Stage/Temp_vs_IR/2m/debut_interpolation/"+ str(sensor)+"_difference.png"
    fig.savefig(fileout,bbox_inches = 'tight')

## Plot de la PT + IR + IR calibrée 

In [ ]:
requested_sensor = ["S1","S2","S3-AIR","S4","S5","S6","S7","S8","S9","S10"]
for sensor in requested_sensor :
    
    # Propriétés graphiques
    fig = plt.figure(figsize=(25,15))
    SMALL_SIZE = 10
    MEDIUM_SIZE = 12
    BIGGER_SIZE = 15
    plt.rc('font', size=BIGGER_SIZE)          # controls default text sizes
    plt.rc('axes', titlesize=BIGGER_SIZE)     # fontsize of the axes title !!! 
    plt.rc('axes', labelsize=BIGGER_SIZE)    # fontsize of the x and y labels
    plt.rc('xtick', labelsize=BIGGER_SIZE)    # fontsize of the tick labels
    plt.rc('ytick', labelsize=BIGGER_SIZE)    # fontsize of the tick labels
    plt.rc('legend', fontsize=BIGGER_SIZE)    # legend fontsize
    plt.rc('figure', titlesize=BIGGER_SIZE)  # fontsize of the figure title
    
    ##Création des figures
    gs = GridSpec(3, 10, figure=fig)
    gs.update(wspace = 0.3, hspace=0.0) # set the spacing between axes.
    ax = fig.add_subplot(gs[0, :])
    
    
    ls_values = [] # listes des valeurs
    ls_time   = [] # listes des temps 
    ls_var    = [] # listes des écarts-types
    rms       = [] # listes des RMS
    
    print(sensor)
    for IR in dict_value_IR :
        ls_values.append(pseudo_temp[sensor][IR]["values"])
        ls_time.append(pseudo_temp[sensor][IR]["time"])
        ls_var.append(pseudo_temp[sensor][IR]["var"])
    
    SLICE = dict_temp_sensor[sensor]
    ## Plot de la température des sondes 
    plottingtemp_single_label_IR_a(SLICE,fig,ax,'T4',200,6)
    ax.set_ylim(-5,30)
    ax3 = ax.twinx() # instantiate a second axes that shares the same x-axis
    color = "tab:red"
    ax.set_ylim(-5,30)
    ax.grid()

    ## Plot de la pseudo Température 
    ax3.plot(ls_time,ls_values,color = color)
    ax3.plot(ls_time,ls_values,'ro',color = color)
    ax3.tick_params(axis = 'y', labelcolor = color)
    ax3.set_ylabel("Pseudo Température", color = color, fontsize = 20)
    ax3.set_ylim(-5,30)
    
    # [ val - sigma , val + sigma]
    
    ls_sigma = [np.array(ls_values)-np.array(ls_var),np.array(ls_values)+np.array(ls_var)]
    
    ## Affichage des valeurs de RMS
    for w in range(len(ls_time)) :
        y_actual    = SLICE.loc[SLICE["Time"]==ls_time[w]]["T4"] 
        y_predicted = ls_values[w]
        rms.append(np.absolute(np.mean(y_actual-y_predicted)))
        ax3.text(ls_time[w],ls_values[w]+0.5,str(rms[w])[0:5],fontsize = 15) # valeurs RMS
  
    ## Affichage de la valeur de RMSE (une valeur)
    ls_slice_value = []
    for time in ls_time : # compare uniquement les pseudo-temp et les valeurs de sonde 
        ls_slice_value.append(float(SLICE["T4"].loc[SLICE['Time'] == time]))
    
    RMSE = mean_squared_error(ls_slice_value,ls_values,squared=False)
    ax3.text(SLICE["Time"].iloc[-8],28,'RMSE = ' + str(RMSE)[0:5],fontsize = 15) # valeurs RMSE
    
    # Interval de confiance +/- sigma 
    ax3.plot(ls_time,ls_sigma[0],'k',linestyle='--',label = '- sigma') # value - sigma
    ax3.plot(ls_time,ls_sigma[1],'k',linestyle='--',label = '+ sigma') # value + sigma
    ax3.legend(loc='upper right', bbox_to_anchor=(1, 0.80))
    
    # plot des target IR Calibrée / Non Calibrée
    for k,IR in enumerate(dict_IR) :
        
        ax2 = fig.add_subplot(gs[1, k])
        window_IR = dict_IR[IR]["dict_windows_param"][sensor]["all_band"]
        window_transform_IR = dict_IR[IR]["dict_windows_param"][sensor]["win_transform"]
        extent = plotting_extent(window_IR,window_transform_IR)

        sub_fig = ax2.imshow(window_IR,extent = extent,cmap ='viridis')
        ax2.set_title(IR + ' non calibrée',fontsize = 10,pad = 25)
        cbar = fig.colorbar(sub_fig,ax=ax2,fraction=0.046, pad=0.04)
        cbar.minorticks_on()
        cbar.ax.tick_params(labelsize=10)
        ax2.set_xticks([ ])
        ax2.set_yticks([ ])
        
        # Valeurs calibrées 
        ax4 = fig.add_subplot(gs[2, k])
        window_IR_calibration = dict_IR[IR]["dict_windows_param"][sensor]["calibration"]
        window_transform_IR_calibration = dict_IR[IR]["dict_windows_param"][sensor]["win_transform"]
        extent_calibration = plotting_extent(window_IR_calibration,window_transform_IR_calibration)

        sub_fig = ax4.imshow(window_IR_calibration,extent = extent_calibration,cmap ='viridis')
        ax4.set_title(IR + ' CALIBREE',fontsize = 10,pad = 25)
        cbar = fig.colorbar(sub_fig,ax=ax4,fraction=0.046, pad=0.04)
        cbar.minorticks_on()
        cbar.ax.tick_params(labelsize=10)
        ax4.set_xticks([ ])
        ax4.set_yticks([ ])
    plt.subplots_adjust(wspace=0.01,hspace=-0.7)
    ax.set_xticklabels([str(time.hour)+':'+str(time.minute) for time in dict_rms[sensor]["Time"]])
    ax.set_xlabel('Time (heures:minutes)' + ' 2019-04-19' )
    ax.set_ylabel('Temp [C]')
    fileout =  "C:/Users/Alexandre/Desktop/Cours/Stage/Temp_vs_IR/2m/calibration/"+ str(sensor)+"_calibration.png"
    fig.savefig(fileout,bbox_inches = 'tight' )
    

## Génération de l'IR 7h15 pour toutes les sondes

In [ ]:
from matplotlib.gridspec import GridSpec

fig = plt.figure(figsize=(32,15))

gs = GridSpec(4, 10, figure=fig)
gs.update(wspace = 0.5, hspace=0.0) # set the spacing between axes.

            
for k,sensor in enumerate(dict_sensor) :

    ax2 = fig.add_subplot(gs[0, k])
    window_IR_1 = dict_IR["IR_6H55"]["dict_windows_param"][sensor]["calibration"]
    window_transform_IR = dict_IR["IR_6H55"]["dict_windows_param"][sensor]["win_transform"]
    extent = plotting_extent(window_IR_1,window_transform_IR)
    MIN = np.nanmin(dict_IR["IR_6H55"]["dict_windows_param"][sensor]["calibration"])
    MAX = np.nanmax(dict_IR["IR_6H55"]["dict_windows_param"][sensor]["calibration"])
    
    sub_fig = ax2.imshow(window_IR_1,extent = extent,cmap ='viridis',vmin = MIN,vmax = MAX)
    ax2.set_title(sensor + ' ' +"IR_6H55", pad = 0.5 )
    ax2.set_xticks([ ])
    ax2.set_yticks([ ])
    cbar = fig.colorbar(sub_fig,ax=ax2,fraction=0.046, pad=0.04)
    cbar.minorticks_on()
    
    
    
    ax3 = fig.add_subplot(gs[1, k])
    window_IR_interpoled = dict_IR_calibration['IR_7H15']["calibration_array"][sensor]
    window_transform_IR = dict_IR["IR_6H55"]["dict_windows_param"][sensor]["win_transform"]
    extent = plotting_extent(window_IR_interpoled,window_transform_IR)
    sub_fig2 = ax3.imshow(window_IR_interpoled,extent = extent,cmap ='viridis',vmin = MIN,vmax = MAX)
    cbar = fig.colorbar(sub_fig2,ax=ax3,fraction=0.046, pad=0.04)
    cbar.minorticks_on()
    ax3.set_title(sensor + ' ' +"IR_7H15", pad = 0.5 )
    ax3.set_xticks([ ])
    ax3.set_yticks([ ])
    
    ax5 = fig.add_subplot(gs[2, k])
    window_IR_interpoled = dict_IR_calibration['IR_7H15_2']["calibration_array"][sensor]
    window_transform_IR = dict_IR["IR_6H55"]["dict_windows_param"][sensor]["win_transform"]
    extent = plotting_extent(window_IR_interpoled,window_transform_IR)
    sub_fig2 = ax5.imshow(window_IR_interpoled,extent = extent,cmap ='viridis',vmin = MIN,vmax = MAX)
    cbar = fig.colorbar(sub_fig2,ax=ax5,fraction=0.046, pad=0.04)
    cbar.minorticks_on()
    ax5.set_title(sensor + ' ' +"IR_7H15_2", pad = 0.5 )
    ax5.set_xticks([ ])
    ax5.set_yticks([ ])
    
    ax4 = fig.add_subplot(gs[3, k])
    window_IR = dict_IR["IR_7H29"]["dict_windows_param"][sensor]["calibration"]
    window_transform_IR = dict_IR['IR_7H29']["dict_windows_param"][sensor]["win_transform"]
    extent = plotting_extent(window_IR,window_transform_IR)
    sub_fig = ax4.imshow(window_IR,extent = extent,cmap ='viridis',vmin = MIN,vmax = MAX)
    ax4.set_title(sensor + ' ' +"IR_7H29" , pad = 0.5 )
    ax4.set_xticks([ ])
    ax4.set_yticks([ ])
    cbar = fig.colorbar(sub_fig,ax=ax4,fraction=0.046, pad=0.04)
    cbar.minorticks_on()

#fileout =  "C:/Users/Alexandre/Desktop/Cours/Stage/Temp_vs_IR/2m/Normalisation commune/"+ str(sensor)+".png"
#fig.savefig(fileout,bbox_inches = 'tight')
            
            
            

In [ ]:
np.nanmax(dict_IR["IR_6H55"]["dict_windows_param"]["S1"]["calibration"])

## Elimination des valeurs proche de la plaque en np.NaN : Visualisation
### Sauf S3-AIR et S10

##### plot de toutes les IR pour une sonde

In [ ]:
requested_sensor = ["S1"]

fig1 = plt.figure(figsize=(32,25))

gs1 = GridSpec(1, 10, figure=fig)
gs1.update(wspace = 0.5, hspace=0.0) # set the spacing between axes.

for sensor in requested_sensor :
    for k,IR in enumerate(dict_IR) :
        ax2 = fig1.add_subplot(gs1[0, k])

        window_IR = dict_IR[IR]["dict_windows_param"][sensor]["all_band"]
        window_transform_IR = dict_IR[IR]["dict_windows_param"][sensor]["win_transform"]
        extent = plotting_extent(window_IR,window_transform_IR)

        sub_fig = ax2.imshow(window_IR,extent = extent,cmap ='viridis')
        ax2.set_title(IR,pad = 20)
        fig1.colorbar(sub_fig,ax=ax2,fraction=0.046, pad=0.04)
        

        # Plot des targets
        TARGET = sensor_coord.loc[sensor_coord["SensorName"]==sensor]
        c_1 = dict_suppress_plaque[IR][sensor]["x"].iloc[0]
        c_2 = dict_suppress_plaque[IR][sensor]["x"].iloc[-1]
        X = np.array(TARGET["x"],dtype = float)
        Y = np.array(TARGET["y"],dtype=float)
        #circle1 = plt.Circle((X,Y),(c_1-X),color='b', fill=False)
        #circle2 = plt.Circle((X,Y),(c_2-X),color='r', fill=False)
        #ax2.add_patch(circle1)
        #ax2.add_patch(circle2)
        ax2.set_xticks([ ])
        ax2.set_yticks([ ])
        #VALUE_TO_SUPPRESS = dict_suppress_plaque[IR][sensor]["value"].describe()['25%']
        #mask_test = dict_suppress_plaque[IR][sensor]["value"] < VALUE_TO_SUPPRESS
        #masked_value = dict_suppress_plaque[IR][sensor].loc[mask_test]
        #ax2.plot(masked_value["x"],masked_value["y"],"ro")

##### Une IR et une sonde ( visualisation de la calibration )

In [ ]:
sensor = "S2"
IR = "IR_6H55"

fig_test,ax_test = plt.subplots()

window_IR_1 = dict_IR[IR]["dict_windows_param"][sensor]["calibration"]
window_transform_IR = dict_IR[IR]["dict_windows_param"][sensor]["win_transform"]
extent = plotting_extent(window_IR_1,window_transform_IR)
sub_fig = ax_test.imshow(window_IR_1,extent = extent,cmap ='viridis')
cbar = fig_test.colorbar(sub_fig,fraction=0.046, pad=0.04)
cbar.minorticks_on()

# Plot des targets
TARGET = sensor_coord.loc[sensor_coord["SensorName"]==sensor]
c_1 = dict_suppress_plaque[IR][sensor]["x"].iloc[0]
c_2 = dict_suppress_plaque[IR][sensor]["x"].iloc[-1]
X = np.array(TARGET["x"],dtype = float)
Y = np.array(TARGET["y"],dtype=float)
circle1 = plt.Circle((X,Y),(c_1-X),color='b', fill=False)
circle2 = plt.Circle((X,Y),(c_2-X),color='r', fill=False)
ax_test.add_patch(circle1)
ax_test.add_patch(circle2)
VALUE_TO_SUPPRESS = dict_suppress_plaque[IR][sensor]["value"].describe()['50%']
mask_test = dict_suppress_plaque[IR][sensor]["value"] < VALUE_TO_SUPPRESS
masked_value = dict_suppress_plaque[IR][sensor].loc[mask_test]
#ax_test.plot(masked_value["x"],masked_value["y"],"ro")
 


# Boxplot + label de clustering

## Visible

In [ ]:
from matplotlib.gridspec import GridSpec
from sklearn.metrics import mean_squared_error
requested_sensor = ["S1"]

#n = dict_windows_param_cluster_hsv[sensor]["all_band"].max()

n = 7 # !!!!!!!!!!!!!!!!!!

for sensor in requested_sensor :
    
    # Propriétés graphiques
    fig = plt.figure(figsize=(30,22))
    SMALL_SIZE = 10
    MEDIUM_SIZE = 12
    BIGGER_SIZE = 15
    plt.rc('font', size=BIGGER_SIZE)          # controls default text sizes
    plt.rc('axes', titlesize=BIGGER_SIZE)     # fontsize of the axes title !!! 
    plt.rc('axes', labelsize=BIGGER_SIZE)    # fontsize of the x and y labels
    plt.rc('xtick', labelsize=BIGGER_SIZE)    # fontsize of the tick labels
    plt.rc('ytick', labelsize=BIGGER_SIZE)    # fontsize of the tick labels
    plt.rc('legend', fontsize=BIGGER_SIZE)    # legend fontsize
    plt.rc('figure', titlesize=BIGGER_SIZE)  # fontsize of the figure title
    
    ##Création des figures
    gs = GridSpec(2, 10, figure=fig)
    gs.update(wspace = 0.0, hspace=0.0) # set the spacing between axes.
    
    
    #### boxplot
    ax = fig.add_subplot(gs[0, :])
    cmap = sns.color_palette("turbo",n)
    data = dict_sensor[sensor]
    
    #ax.set_ylim(0,1)
    sns.boxplot(x='IR',y="band N",
               data = data,hue = "clustering_label_vis",palette = cmap,ax = ax, hue_order = [0,1,2,3,4,5,6])

    sns.set_theme(style="ticks",font_scale = 2)
    sns.despine(offset=0, trim=True)
    LONGUEUR = dict_IR["IR_6H55"]["dict_windows_param"][sensor]["LONGUEUR"]

    ax.set_ylabel("Signal IR normalisée",fontsize = 20)
    plt.title(sensor + ' '+str(LONGUEUR)+ 'm patch' + " normalisée")
        
    ## plot des labels de clustering 
    #visible
    ax4 = fig.add_subplot(gs[1, 0:2])
    #n = dict_windows_param_cluster[sensor]["all_band"].max()
    ole=cm.get_cmap('turbo', n)
    
    window_cluster = dict_windows_param_cluster[sensor]["all_band"]
    window_cluster_transform = dict_windows_param_cluster[sensor]["win_transform"]
    extent = plotting_extent(window_cluster,window_cluster_transform)
    sub_fig = ax4.imshow(window_cluster,extent = extent,cmap = ole,vmin = 0,vmax = n-1)
    ax4.set_title(sensor + " cluster sur la bande visible",fontsize = 20,pad = 20)
    fig.colorbar(sub_fig,ax=ax4,fraction=0.046, pad=0.04)
    
    #hsv
    ax5 = fig.add_subplot(gs[1, 7:-1])
    window_hsv = dict_windows_param_cluster_hsv[sensor]["all_band"]
    window_transform_hsv = dict_windows_param_cluster_hsv[sensor]["win_transform"]
    extent = plotting_extent(window_hsv,window_transform_hsv)
    sub_fig = ax5.imshow(window_hsv,extent = extent,cmap = ole,vmin = 0,vmax = n-1)
    ax5.set_title(sensor + " cluster sur la bande HSV",fontsize = 20,pad = 20)
    fig.colorbar(sub_fig,ax=ax5,fraction=0.046, pad=0.04)
    
    ## plot du patch visible
    ax6 = fig.add_subplot(gs[1, 3:6])
    window_vis = dict_windows_param[sensor]["all_band"]
    window_transform_vis = dict_windows_param[sensor]["win_transform"]
    show(window_vis,transform = window_transform_vis,ax = ax6)
    ax6.set_title(' ',fontsize = 20,pad = 20)
    
    ## paramètres d'affichage
    modification = '_calibree'
    fileout =  "C:/Users/Alexandre/Desktop/Cours/Stage/Temp_vs_IR/2m/clustering/visible/"+ str(sensor)+".png"
    fig.savefig(fileout,bbox_inches = 'tight' )

# Figure Rapport

In [ ]:
from matplotlib.gridspec import GridSpec
from sklearn.metrics import mean_squared_error

requested_sensor = ["S1"]

#n = dict_windows_param_cluster_hsv["all_band"].max()

n = 7 # nombre de cluster

for sensor in dict_sensor :
    ##Création des figures
    fig = plt.figure(figsize=(30,22))
    gs = GridSpec(2, 10, figure=fig)
    gs.update(wspace = 0.0, hspace=0.0) # set the spacing between axes.
    
 ## plot des labels de clustering 
    #visible
    ax4 = fig.add_subplot(gs[1, 0:2])
    #n = dict_windows_param_cluster[sensor]["all_band"].max()
    ole=cm.get_cmap('turbo', n)
    
    window_cluster = dict_windows_param_cluster[sensor]["all_band"]
    window_cluster_transform = dict_windows_param_cluster[sensor]["win_transform"]
    extent = plotting_extent(window_cluster,window_cluster_transform)
    sub_fig = ax4.imshow(window_cluster,extent = extent,cmap = ole,vmin = 0,vmax = n-1)
    ax4.set_title(sensor + " cluster sur la bande visible",fontsize = 20,pad = 20)
    fig.colorbar(sub_fig,ax=ax4,fraction=0.046, pad=0.04)
    
    #hsv
    ax5 = fig.add_subplot(gs[1, 7:-1])
    window_hsv = dict_windows_param_cluster_hsv[sensor]["all_band"]
    window_transform_hsv = dict_windows_param_cluster_hsv[sensor]["win_transform"]
    extent = plotting_extent(window_hsv,window_transform_hsv)
    sub_fig = ax5.imshow(window_hsv,extent = extent,cmap = ole,vmin = 0,vmax = n-1)
    ax5.set_title(sensor + " cluster sur la bande HSV",fontsize = 20,pad = 20)
    fig.colorbar(sub_fig,ax=ax5,fraction=0.046, pad=0.04)
    
    ## plot du patch visible
    ax6 = fig.add_subplot(gs[1, 3:6])
    window_vis = dict_windows_param[sensor]["all_band"]
    window_transform_vis = dict_windows_param[sensor]["win_transform"]
    show(window_vis,transform = window_transform_vis,ax = ax6)
    ax6.set_title("Fenêtrage autour de la sonde "+sensor,fontsize = 20,pad = 20)
    
    ## paramètres d'affichage
    fileout =  "C:/Users/Alexandre/Desktop/Cours/Stage/Modification/Fenêtrage autour des sondes/"+ str(sensor)+".png"
    fig.savefig(fileout,bbox_inches = 'tight' )

In [ ]:
dict_windows_param

In [ ]:
from matplotlib.gridspec import GridSpec
from sklearn.metrics import mean_squared_error
requested_sensor = ["S1"]

#n = dict_windows_param_cluster_hsv[sensor]["all_band"].max()

n = 7 # !!!!!!!!!!!!!!!!!!

for sensor in requested_sensor :
    
    # Propriétés graphiques
    fig,ax = plt.subplots(1,1,figsize=(7,5))
    SMALL_SIZE = 10
    MEDIUM_SIZE = 12
    BIGGER_SIZE = 15
    plt.rc('font', size=BIGGER_SIZE)          # controls default text sizes
    plt.rc('axes', titlesize=BIGGER_SIZE)     # fontsize of the axes title !!! 
    plt.rc('axes', labelsize=BIGGER_SIZE)    # fontsize of the x and y labels
    plt.rc('xtick', labelsize=BIGGER_SIZE)    # fontsize of the tick labels
    plt.rc('ytick', labelsize=BIGGER_SIZE)    # fontsize of the tick labels
    plt.rc('legend', fontsize=BIGGER_SIZE)    # legend fontsize
    plt.rc('figure', titlesize=BIGGER_SIZE)  # fontsize of the figure title
    
    ## plot du patch visible
    
    window_vis = dict_windows_param[sensor]["all_band"]
    window_transform_vis = dict_windows_param[sensor]["win_transform"]
    extent = plotting_extent(window_IR,window_transform_IR)
    show(window_vis,transform = window_transform_vis,ax = ax)
    #ax[0].set_title(' ',fontsize = 20,pad = 20)
    
    ## paramètres d'affichage
    modification = '_visible'
    fileout =  "C:/Users/Alexandre/Desktop/Cours/Stage/Rapport/Prediction/patch"+modification+".png"
    fig.savefig(fileout,bbox_inches = 'tight' )
    
    fig,ax = plt.subplots(1,1,figsize=(7,5))
    
    ## plot du patch IR
    window_IR = dict_IR["IR_6H55"]["dict_windows_param"][sensor]["all_band"]
    window_transform_IR = dict_IR["IR_6H55"]["dict_windows_param"][sensor]["win_transform"]
    extent = plotting_extent(window_IR,window_transform_IR)
    sub_fig = ax.imshow(window_IR,extent = extent, cmap = "plasma")
    ax.set_title('',fontsize = 20,pad = 20)
    cbar = fig.colorbar(sub_fig,ax=ax,fraction=0.046, pad=0.04)
    cbar.set_label(label = "Signal IR",size = 18)
    
    
    ## paramètres d'affichage
    modification = '_IR'
    fileout =  "C:/Users/Alexandre/Desktop/Cours/Stage/Rapport/Prediction/patch"+modification+".png"
    fig.savefig(fileout,bbox_inches = 'tight' )

## HSV

In [ ]:
from matplotlib.gridspec import GridSpec
from sklearn.metrics import mean_squared_error
requested_sensor = ["S1"]

#n = dict_windows_param_cluster_hsv[sensor]["all_band"].max()

n = 7 # !!!!!!!!!!!!!!!!!!

for sensor in dict_sensor :
    
    # Propriétés graphiques
    fig = plt.figure(figsize=(30,22))
    SMALL_SIZE = 10
    MEDIUM_SIZE = 12
    BIGGER_SIZE = 15
    plt.rc('font', size=BIGGER_SIZE)          # controls default text sizes
    plt.rc('axes', titlesize=BIGGER_SIZE)     # fontsize of the axes title !!! 
    plt.rc('axes', labelsize=BIGGER_SIZE)    # fontsize of the x and y labels
    plt.rc('xtick', labelsize=BIGGER_SIZE)    # fontsize of the tick labels
    plt.rc('ytick', labelsize=BIGGER_SIZE)    # fontsize of the tick labels
    plt.rc('legend', fontsize=BIGGER_SIZE)    # legend fontsize
    plt.rc('figure', titlesize=BIGGER_SIZE)  # fontsize of the figure title
    
    ##Création des figures
    gs = GridSpec(2, 10, figure=fig)
    gs.update(wspace = 0.0, hspace=0.0) # set the spacing between axes.
    
    
    #### boxplot
    ax = fig.add_subplot(gs[0, :])
    cmap = sns.color_palette("turbo",n)
    data = dict_sensor[sensor]
    
    #ax.set_ylim(0,1)
    sns.boxplot(x='IR',y="band N",
               data = data,hue = "clustering_label_hsv",palette = cmap,ax = ax, hue_order = [0,1,2,3,4,5,6])

    sns.set_theme(style="ticks",font_scale = 2)
    sns.despine(offset=0, trim=True)
    LONGUEUR = dict_IR["IR_6H55"]["dict_windows_param"][sensor]["LONGUEUR"]

    ax.set_ylabel("Signal IR normalisée",fontsize = 20)
    plt.title(sensor + ' '+str(LONGUEUR)+ 'm patch' + " normalisée")
        
    ## plot des labels de clustering 
    #visible
    ax4 = fig.add_subplot(gs[1, 0:2])
    #n = dict_windows_param_cluster[sensor]["all_band"].max()
    ole=cm.get_cmap('turbo', n)
    
    window_cluster = dict_windows_param_cluster[sensor]["all_band"]
    window_cluster_transform = dict_windows_param_cluster[sensor]["win_transform"]
    extent = plotting_extent(window_cluster,window_cluster_transform)
    sub_fig = ax4.imshow(window_cluster,extent = extent,cmap = ole,vmin = 0,vmax = n-1)
    ax4.set_title(sensor + " cluster sur la bande visible",fontsize = 20,pad = 20)
    fig.colorbar(sub_fig,ax=ax4,fraction=0.046, pad=0.04)
    
    #hsv
    ax5 = fig.add_subplot(gs[1, 7:-1])
    window_hsv = dict_windows_param_cluster_hsv[sensor]["all_band"]
    window_transform_hsv = dict_windows_param_cluster_hsv[sensor]["win_transform"]
    extent = plotting_extent(window_hsv,window_transform_hsv)
    sub_fig = ax5.imshow(window_hsv,extent = extent,cmap = ole,vmin = 0,vmax = n-1)
    ax5.set_title(sensor + " cluster sur la bande HSV",fontsize = 20,pad = 20)
    fig.colorbar(sub_fig,ax=ax5,fraction=0.046, pad=0.04)
    
    ## plot du patch visible
    ax6 = fig.add_subplot(gs[1, 3:6])
    window_vis = dict_windows_param[sensor]["all_band"]
    window_transform_vis = dict_windows_param[sensor]["win_transform"]
    show(window_vis,transform = window_transform_vis,ax = ax6)
    ax6.set_title(' ',fontsize = 20,pad = 20)
    
    ## paramètres d'affichage
    modification = '_calibree'
    fileout =  "C:/Users/Alexandre/Desktop/Cours/Stage/Temp_vs_IR/2m/clustering/hsv/"+ str(sensor)+".png"
    fig.savefig(fileout,bbox_inches = 'tight' )

### Boxplot

In [ ]:
from matplotlib.gridspec import GridSpec
from sklearn.metrics import mean_squared_error
requested_sensor = ["S1","S2","S3-AIR","S4","S5","S6","S7","S8","S9","S10"]
#n = dict_windows_param_cluster_hsv[sensor]["all_band"].max()
n = 6
for sensor in requested_sensor :
    
    # Propriétés graphiques
    fig = plt.figure(figsize=(50,25))
    SMALL_SIZE = 10
    MEDIUM_SIZE = 12
    BIGGER_SIZE = 15
    plt.rc('font', size=BIGGER_SIZE)          # controls default text sizes
    plt.rc('axes', titlesize=BIGGER_SIZE)     # fontsize of the axes title !!! 
    plt.rc('axes', labelsize=BIGGER_SIZE)    # fontsize of the x and y labels
    plt.rc('xtick', labelsize=BIGGER_SIZE)    # fontsize of the tick labels
    plt.rc('ytick', labelsize=BIGGER_SIZE)    # fontsize of the tick labels
    plt.rc('legend', fontsize=BIGGER_SIZE)    # legend fontsize
    plt.rc('figure', titlesize=BIGGER_SIZE)  # fontsize of the figure title
    
    ##Création des figures
    gs = GridSpec(2, 10, figure=fig)
    gs.update(wspace = 0.0, hspace=0.0) # set the spacing between axes.
    ax = fig.add_subplot(gs[0, :])
    
    ## boxplot
    cmap = sns.color_palette("turbo",n+1)
    data = dict_sensor[sensor]
    #ax.set_ylim(0,1)
    sns.boxplot(x='IR',y="calibration",
               data = data,hue = "clustering_label_vis",palette = cmap,ax = ax, hue_order = [0,1,2,3,4,5,6])

    sns.set_theme(style="ticks",font_scale = 2)
    sns.despine(offset=0, trim=True)
    LONGUEUR = dict_IR["IR_6H55"]["dict_windows_param"][sensor]["LONGUEUR"]

    ax.set_ylabel("Signal IR, valeurs sans normalisation",fontsize = 15)
    plt.title(sensor + ' '+str(LONGUEUR)+ 'm patch')
    
    ## paramètres d'affichage
    modification = '_calibree'
    fileout =  "C:/Users/Alexandre/Desktop/Cours/Stage/Temp_vs_IR/2m/"+ str(sensor)+modification+".png"
    #fig.savefig(fileout)

In [ ]:
dict_IR["IR_6H55"]["dict_windows_param"][sensor]["calibration"].min()

In [ ]:
from matplotlib.gridspec import GridSpec
from sklearn.metrics import mean_squared_error
requested_sensor = ["S1","S2","S3-AIR","S4","S5","S6","S7","S8","S9","S10"]
#n = dict_windows_param_cluster_hsv[sensor]["all_band"].max()
n = 6
for sensor in requested_sensor :
    
    # Propriétés graphiques
    fig = plt.figure(figsize=(50,25))
    SMALL_SIZE = 10
    MEDIUM_SIZE = 12
    BIGGER_SIZE = 15
    plt.rc('font', size=BIGGER_SIZE)          # controls default text sizes
    plt.rc('axes', titlesize=BIGGER_SIZE)     # fontsize of the axes title !!! 
    plt.rc('axes', labelsize=BIGGER_SIZE)    # fontsize of the x and y labels
    plt.rc('xtick', labelsize=BIGGER_SIZE)    # fontsize of the tick labels
    plt.rc('ytick', labelsize=BIGGER_SIZE)    # fontsize of the tick labels
    plt.rc('legend', fontsize=BIGGER_SIZE)    # legend fontsize
    plt.rc('figure', titlesize=BIGGER_SIZE)  # fontsize of the figure title
    
    ##Création des figures
    gs = GridSpec(2, 10, figure=fig)
    gs.update(wspace = 0.0, hspace=0.0) # set the spacing between axes.
    ax = fig.add_subplot(gs[0, :])
    
    ## boxplot
    cmap = sns.color_palette("turbo",n+1)
    data = dict_sensor[sensor]
    #ax.set_ylim(0,1)
    sns.boxplot(x='IR',y="calibration",
               data = data,hue = "clustering_label_vis",palette = cmap,ax = ax, hue_order = [0,1,2,3,4,5,6])

    sns.set_theme(style="ticks",font_scale = 2)
    sns.despine(offset=0, trim=True)
    LONGUEUR = dict_IR["IR_6H55"]["dict_windows_param"][sensor]["LONGUEUR"]

    ax.set_ylabel("Signal IR, valeurs sans normalisation",fontsize = 15)
    plt.title(sensor + ' '+str(LONGUEUR)+ 'm patch')
    
    ## paramètres d'affichage
    modification = '_calibree'
    fileout =  "C:/Users/Alexandre/Desktop/Cours/Stage/Temp_vs_IR/2m/"+ str(sensor)+modification+".png"
    #fig.savefig(fileout)

## ANNEXE

# Suppression de l'effet de la plaque et mise à jour de Dict_IR avec np.NaN

In [ ]:
def supress_circle_value (ls_path_tif,dict_IR,r2 = 0.5) :
    """
    Création d'un dictionnaire classé par IR contenant toutes les données comprise entre un rayon r1 et r2, 
    """
    
    dict_suppress_plaque = {}

    for i,IR in enumerate(dict_IR) :
        dict_value_sensor = {}
        with rio.open(ls_path_tif[i]) as dataset : # ouverture du fichier obligatoire pour recup la valeur
            for sensor in dict_IR[IR]["dict_windows"] :
                if sensor in ["S10","S3-AIR"] : # sonde sans plaque
                    print(sensor,"Ne présente pas de plaque")
                    
                else:
                    #print(sensor)
                    res = dict_IR[IR]["dict_windows_param"][sensor]["resolution"] # resolution [x,y]
                    sensor_dataframe = dict_IR[IR]["dict_windows"][sensor]
                    mid_indice = (dict_IR[IR]["dict_windows_param"][sensor]['win'].width) # les indices commencent à 0 !

                    mid_x = sensor_dataframe.iloc[len(sensor_dataframe)//2]["x"] 
                    mid_y = sensor_dataframe.iloc[(mid_indice//2)-1]["y"]# la valeur du milieu est //2 

                    #list des valeurs autour de [0, 2pi] , mouvement anti-horaire
                    theta = np.linspace(0,np.pi*2,round(np.pi*2/res[0]))
                    ls_x = np.empty([1]) ; ls_y = np.empty([1])

                    #recherche de points entre deux rayon r1 et r2
                    r1 = 0.0

                    for THETA in theta :
                        #
                        x1 = mid_x + r1*np.cos(THETA)
                        y1 = mid_y + r1*np.sin(THETA)

                        x2 = mid_x + r2*np.cos(THETA)
                        y2 = mid_y + r2*np.sin(THETA)

                        ls_x = np.append(ls_x,np.linspace(x1,x2,round((r2-r1)/res[0]))) 
                        ls_y = np.append(ls_y,np.linspace(y1,y2,round((r2-r1)/res[1])))

                    ls_x = np.delete(ls_x,0) ; ls_y = np.delete(ls_y,0)

                    dict_IR[IR]["dict_windows_param"][sensor]['win'].width # résolution -> nb de pixel == nb d'indice 

                    ls_value = []
                    for k in range(len(ls_x)) :

                        for value in dataset.sample([(float(ls_x[k]),float(ls_y[k]))]) : 
                            ls_value.append(float(value))

                    dict_value_sensor[sensor] = pd.DataFrame(np.array([ls_x,ls_y,ls_value],dtype = float).T,columns = 
                                                             ["x","y","value"])
            
                    dict_suppress_plaque[IR] = dict_value_sensor
            
                    ##Conversion des pixels proche de la plaque en np.NAN
                    DICT_SENSOR = dict_sensor[sensor].loc[dict_sensor[sensor]["IR"] == IR] #dictionnaire des sensors
                    DICT_SS_PLAQUE = dict_suppress_plaque[IR][sensor] # dictionnaire sans les plaques

                    #On pourrait recupérer les résultats de ls_x et ls_y à modifier
                    condition  = (DICT_SENSOR["y"]>DICT_SS_PLAQUE["y"].min()) & (DICT_SENSOR["y"]<DICT_SS_PLAQUE["y"].max())
                    condition2 = (DICT_SENSOR["x"]>DICT_SS_PLAQUE["x"].min()) & (DICT_SENSOR["x"]<DICT_SS_PLAQUE["x"].max())
                    #set to NaN dans le DataFrame
                    DICT_SENSOR.loc[condition & condition2,"origin band" : "band N"] = np.NAN

                    #Modification des valeurs directement dans l'array correspondant

                    ls_x_row , ls_y_col = [] , [] #listes des valeurs x et y 
                    dataframe_position = DICT_SENSOR.loc[condition & condition2][["x","y"]].values #conversion df -> list

                    print(ls_path_tif[0],sensor) #print de contrôle
                    for x_val,y_val in dataframe_position :
                        x_row,y_col = dataset.index(x_val,y_val) # valueur d'indice de chaque position
                        win_propiety = dict_IR[IR]["dict_windows_param"][sensor]["win"]
                        #conversion des indices de l'IR -> indice dans la fenêtre
                        x_row = np.absolute(win_propiety.row_off-x_row) 
                        y_col = np.absolute(win_propiety.col_off-y_col)

                        #set to NaN dans l'array
                        dict_IR[IR]["dict_windows_param"][sensor]["all_band"][x_row,y_col] = np.NAN

    print(dict_IR["IR_6H55"]["dict_windows_param"]["S2"]["all_band"][x_row-1: x_row+12,y_col-5 : y_col])
        
    return dict_suppress_plaque

In [ ]:
dict_suppress_plaque = supress_circle_value (ls_path_tif,dict_IR,0.3)

# Mise à jour de dict_sensor et dict_IR avec les données de calibration

### Calibration des IR à l'aide des valeurs de RMS

In [ ]:
def calibration_to_DataFrame(dict_sensor,dict_IR,dict_rms):
    """
    Mise à jour de deux dictionnaires avec les valeurs de calibration
    """

    for sensor in dict_sensor :
        for k,IR in enumerate(dict_IR) :
            
            # calibration sous forme d'array pour affichage imshow()
            calibration = dict_rms[sensor]['RMS'][k]
            IR_calibration = dict_IR[IR]["dict_windows_param"][sensor]["all_band"] + calibration
            
            dict_IR[IR]["dict_windows_param"][sensor]["calibration"] = IR_calibration
            dict_IR[IR]["dict_windows_param"][sensor]["calibration_value"] = calibration
            # usage d'un mask pour repérer les valeurs de chaque IR 
            mask = dict_sensor[sensor]["IR"].str.startswith(str(IR))
            
            # valeur unique de calibration par sensor par IR, situé dans le dict dict_IR qui contient toutes 
            # les données brute/dataframe classées par IR 
            cal = dict_IR[IR]["dict_windows_param"][sensor]["calibration_value"]
            
            print(sensor,IR,'Calibration value :',str(cal)[0:7])
            
            #Assigniation des donnees au dataframe 
            dict_sensor[sensor].loc[mask,"calibration"] = dict_sensor[sensor]['origin band'].add(cal)
            

        print("completed :",sensor)
    print("Les valeurs sous forme d'array sont dispo ici -> dict_IR[IR]['dict_windows_param'][sensor]['calibration']")
    print("La valeur de calibration est dispo ici -> dict_IR[IR]['dict_windows_param'][sensor]['calibration_value']")
    return dict_sensor,dict_IR


In [ ]:
dict_sensor,dict_IR = calibration_to_DataFrame(dict_sensor,dict_IR,dict_rms)

In [ ]:
dict_sensor[sensor]

# Génération de l'IR 7h15 pour toutes les sondes

In [ ]:
def interpolate_calibration_to_DataFrame(dict_sensor,dict_IR,dict_sensor_calibration):
    """
    Création d'un DataFrame par sonde contenant les positions et les des bandes normalisées, réunit sous 
    un dictionnaire
    """

    for sensor in dict_sensor :
        for k,IR in enumerate(dict_IR) :
            # calibration sous forme matricielle pour affichage imshow()
            calibration = dict_rms[sensor]['RMS'][k]
            IR_calibration = dict_IR[IR]["dict_windows_param"][sensor]["all_band"] + calibration
            #dict_calibration_IR[IR] = IR_calibration
            dict_IR[IR]["dict_windows_param"][sensor]["calibration"] = IR_calibration
            dict_IR[IR]["dict_windows_param"][sensor]["calibration_value"] = calibration
            # usage d'un mask pour repérer les valeurs de chaque IR 
            mask = dict_sensor[sensor]["IR"].str.startswith(str(IR))
            
            # valeur unique de calibration par sensor par IR, situé dans le dict dict_IR qui contient toutes 
            # les données brute/dataframe classées par IR 
            cal = dict_IR[IR]["dict_windows_param"][sensor]["calibration_value"]
            
            print(sensor,IR,'Calibration value :',str(cal)[0:7])
            
            #Assigniation des donnees au dataframe 
            dict_sensor[sensor].loc[mask,"calibration"] = dict_sensor[sensor]['origin band'].add(cal)
            

        print("completed :",sensor)
    return dict_sensor,dict_IR


In [ ]:
dict_IR_calibration = {}
IR_calibration = {}
IR_calibration_value = {}
IR_calibration_2 = {}
IR_calibration_value_2 = {}
for sensor in dict_sensor_calibration :
    IR = "IR_6H55"
    interpoled_calibration = (dict_sensor_calibration[sensor]["Interpolate_time"][1],
                              dict_sensor_calibration[sensor]["Interpolate_value"][1])
    print(interpoled_calibration)

    IR_calibration[sensor] = dict_IR[IR]["dict_windows_param"][sensor]["all_band"] + interpoled_calibration[1]
    IR_calibration_value[sensor] = interpoled_calibration[1]
    IR_name = 'IR_'+str(interpoled_calibration[0].hour)+'H'+str(interpoled_calibration[0].minute)
    
    
    IR_2 = "IR_7H29"
    IR_calibration_2[sensor] = dict_IR[IR_2]["dict_windows_param"][sensor]["all_band"] - interpoled_calibration[1]
    IR_calibration_value_2[sensor] = interpoled_calibration[1]
    IR_name_2 = 'IR_'+str(interpoled_calibration[0].hour)+'H'+str(interpoled_calibration[0].minute)+"_2"
    
    

dict_IR_calibration[IR_name] = {"calibration_array" : IR_calibration, 
                                   "Interpolate_value" : IR_calibration_value }

dict_IR_calibration[IR_name_2] = {"calibration_array" : IR_calibration_2, 
                                   "Interpolate_value" : IR_calibration_value_2 }
dict_IR_calibration.keys()
    
    
    

In [ ]:

dict_suppress_plaque = {}
requested_IR = ["IR_6H55"]
requested_sensor = ["S2"]
r2 = 0.3

for i,IR in enumerate(requested_IR) :
    dict_value_sensor = {}
    with rio.open(ls_path_tif[i]) as dataset : # ouverture du fichier obligatoire pour recup la valeur
        for sensor in requested_sensor :
            if sensor in ["S10","S3-AIR"] : # sonde sans plaque
                print(sensor,"Ne présente pas de plaque")

            else :
                #print(sensor)
                res = dict_IR[IR]["dict_windows_param"][sensor]["resolution"] # resolution [x,y]
                sensor_dataframe = dict_IR[IR]["dict_windows"][sensor]
                mid_indice = (dict_IR[IR]["dict_windows_param"][sensor]['win'].width) # les indices commencent à 0 !

                mid_x = sensor_dataframe.iloc[len(sensor_dataframe)//2]["x"] 
                mid_y = sensor_dataframe.iloc[(mid_indice//2)-1]["y"]# la valeur du milieu est //2 

                #list des valeurs autour de [0, 2pi] , mouvement anti-horaire
                theta = np.linspace(0,np.pi*2,round(np.pi*2/res[0]))
                ls_x = np.empty([1]) ; ls_y = np.empty([1])

                #recherche de points entre deux rayon r1 et r2
                r1 = 0.0

                for THETA in theta :
                    #
                    x1 = mid_x + r1*np.cos(THETA)
                    y1 = mid_y + r1*np.sin(THETA)

                    x2 = mid_x + r2*np.cos(THETA)
                    y2 = mid_y + r2*np.sin(THETA)

                    ls_x = np.append(ls_x,np.linspace(x1,x2,round((r2-r1)/res[0]))) 
                    ls_y = np.append(ls_y,np.linspace(y1,y2,round((r2-r1)/res[1])))

                ls_x = np.delete(ls_x,0) ; ls_y = np.delete(ls_y,0)

                dict_IR[IR]["dict_windows_param"][sensor]['win'].width # résolution -> nb de pixel == nb d'indice 
                ls_value = []
                for k in range(len(ls_x)) :

                    for value in dataset.sample([(float(ls_x[k]),float(ls_y[k]))]) : 
                        ls_value.append(float(value))

                dict_value_sensor[sensor] = pd.DataFrame(np.array([ls_x,ls_y,ls_value],dtype = float).T,columns = 
                                                         ["x","y","value"])
                dict_suppress_plaque[IR] = dict_value_sensor
                
                ## Conversion des pixels proche de la plaque en np.NAN
                DICT_SENSOR = dict_sensor[sensor].loc[dict_sensor[sensor]["IR"] == IR]
                DICT_SS_PLAQUE = dict_suppress_plaque[IR][sensor]
                
                #On pourrait recupérer les résultats de ls_x et ls_y à modifier
                condition  = (DICT_SENSOR["y"]>DICT_SS_PLAQUE["y"].min()) & (DICT_SENSOR["y"]<DICT_SS_PLAQUE["y"].max())
                condition2 = (DICT_SENSOR["x"]>DICT_SS_PLAQUE["x"].min()) & (DICT_SENSOR["x"]<DICT_SS_PLAQUE["x"].max())
                #set to NaN dans le DataFrame
                DICT_SENSOR.loc[condition & condition2,"origin band" : "band N"] = np.NAN
                
                #Modification des valeurs directement dans l'array correspondant
                
                ls_x_row , ls_y_col = [] , [] #listes des valeurs x et y 
                dataframe_position = DICT_SENSOR.loc[condition & condition2][["x","y"]].values #conversion df -> list
                
                print(ls_path_tif[0],sensor) #print de contrôle
                for x_val,y_val in dataframe_position :
                    x_row,y_col = dataset.index(x_val,y_val) # valueur d'indice de chaque position
                    win_propiety = dict_IR[IR]["dict_windows_param"][sensor]["win"]
                    #conversion des indices de l'IR -> indice dans la fenêtre
                    x_row = np.absolute(win_propiety.row_off-x_row) 
                    y_col = np.absolute(win_propiety.col_off-y_col)
                    
                    #set to NaN dans l'array
                    dict_IR[IR]["dict_windows_param"][sensor]["all_band"][x_row,y_col] = np.NAN

dict_IR["IR_6H55"]["dict_windows_param"]["S2"]["all_band"][x_row-1: x_row+12,y_col-5 : y_col]
                
                
                


# TEST LSTM

In [ ]:
sensor_temp_complete = SLICE['T4'].values # contient les valeurs de S10

In [ ]:
#convert training data to right shape
trai

In [ ]:
train = SLICE.loc[SLICE["Time"] < "2019-04-14 15:00:00"]['T4'].values.reshape(-1,1) # X
test = SLICE.loc[SLICE["Time"] >= "2019-04-14 15:00:00"]['T4'].values.reshape(-1,1) # Y


In [ ]:
from sklearn.preprocessing import MinMaxScaler
sc = MinMaxScaler(feature_range=(0,1))
train_scaled = sc.fit_transform(train)

In [ ]:
#Create supervized data with 2 hours input and 1 output 
hours = 2 # hours 
n = hours * 4
X_train = train[:-n]
X_test = train[-n:]
Y_train = train[:-n]
Y_test = train[-n:]

In [ ]:
-----------------------------------------------------------------------------------------------------------------

In [ ]:
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM

from sklearn.metrics import mean_squared_error

In [ ]:
from nsepy import get_history
from datetime import datetime

startDate = datetime(2019,1,1)
endDate   = datetime(2020,10,5)

#Fetching the Data
StockData = get_history(symbol = "INFY" , start = startDate , end = endDate)
print(StockData.shape)
StockData.head()


In [ ]:
# Creating a column as date
StockData['TradeDate']=StockData.index

# Plotting the sotck prices
StockData.plot(x='TradeDate', y='Close', kind='line', figsize=(20,6), rot=20)

In [ ]:
# Extracting the closing prices of each day
FullData = StockData[['Close']].values
print(FullData[0:5])

# Feature Scaling for fast training of neural networks
from sklearn.preprocessing import StandardScaler, MinMaxScaler
 
# Choosing between Standardization or normalization
#sc = StandardScaler()
sc=MinMaxScaler()
 
DataScaler = sc.fit(FullData)
X=DataScaler.transform(FullData)
#X=FullData
 
print('### After Normalization ###')
X[0:5]

In [ ]:
#split into samples
X_samples = []
y_samples = []
NumerofRows = len(X)
TimeSteps = 10 # nxt day's Price Prediction is based on last how many day's prices
# On commence au 10e jour 

# Iterate thru the values to create combinations
for i in range(TimeSteps,NumerofRows,1) : # On prend tjrs 10 jours
    # ex i = 10 -> [0 1 2 3 4 5 6 7 8 9]
    # ex i = 11 -> [ 1  2  3  4  5  6  7  8  9 10]
    x_sample = X[i-TimeSteps:i]
    y_sample = X[i]
    X_samples.append(x_sample)
    y_samples.append(y_sample)
len(x_sample)
    


In [ ]:
for i in range(TimeSteps,NumerofRows,1) :
    
    print(i,np.arange(i-TimeSteps,i))

In [ ]:
# Reshape the Input as a 3D (number of samples, Time Steps, Features)
X_data=np.array(X_samples)
X_data=X_data.reshape(X_data.shape[0],X_data.shape[1], 1)
print('\n#### Input Data shape ####')
print(X_data.shape)
 
# We do not reshape y as a 3D data  as it is supposed to be a single column only
y_data=np.array(y_samples)
y_data=y_data.reshape(y_data.shape[0], 1)
print('\n#### Output Data shape ####')
print(y_data.shape)

In [ ]:
X_data=np.array(X_samples)
X_data=X_data.reshape(X_data.shape[0],X_data.shape[1], 1)
print('\n#### Input Data shape ####')
print(X_data.shape)

In [ ]:
X_samples[0]

In [ ]:
------------------------------------------------------------------------------------------------------------------

In [ ]:
file_test = "C:/Users/Alexandre/Desktop/Cours/Stage/airline-passengers.csv"
dataframe = pd.read_csv(file_test,usecols = [1],engine = 'python')
dataset = dataframe.values
dataset = dataset.astype('float32')
#normalize the dataset
scaler = MinMaxScaler(feature_range = (0,1))
dataset = scaler.fit_transform(dataset)


In [ ]:
#split into train and test sets
train_size = int(len(dataset)*0.67) #96
test_size  = len(dataset) - train_size # 48
train, test = dataset[0:train_size,:] , dataset[train_size:len(dataset),:]

In [ ]:
# convert an array of values into a dataset matrix
def create_dataset(dataset, look_back=1):
    dataX, dataY = [], []
    for i in range(len(dataset)-look_back-1):
        a = dataset[i:(i+look_back), 0]
        dataX.append(a)
        dataY.append(dataset[i + look_back, 0])
    return np.array(dataX), np.array(dataY)

In [ ]:
# reshape into X=t and Y=t+1
look_back = 1
trainX, trainY = create_dataset(train, look_back)
testX, testY = create_dataset(test, look_back)
# reshape input to be [samples,time steps, features]
trainX = np.reshape(trainX,(trainX.shape[0], 1, trainX.shape[1]))
testX = np.reshape(testX, (testX.shape[0], 1, testX.shape[1]))

In [ ]:
look_back

In [ ]:
# create and fit the LSTM network
model = Sequential()
model.add(LSTM(unit = 10, input_shape =(1,look_back) ) )
model.add(Dense(1))
model.compile(loss = 'mean_squared_error', optimizer = 'adam')
model.fit(trainX, trainY, epochs)

In [ ]:
XX = np.array(train[0:-2]).reshape(-1,1)

In [ ]:
XX[-7:]

In [ ]:
trainX[-7:]

In [ ]:
data["clustering_label"].min()

In [ ]:
sns.color_palette("turbo",as_cmap=True)

In [ ]:
sns.color_palette('turbo',7)

In [ ]:
with sns.color_palette("husl") as cmap :
        data = dict_sensor[sensor]
        
        ax = fig.add_subplot(gs[0, :])
        
        ax.set_ylim(0,1)
        sns.boxplot(x='IR',y="band N",
                   data = data,palette = cmap,ax = ax )
        sns.set_theme(style="ticks",font_scale = 2)
        sns.despine(offset=10, trim=True)
        LONGUEUR = dict_IR["IR_6H55"]["dict_windows_param"]["S10"]["LONGUEUR"]
        
        ax.set_ylabel("Normalisation commune",fontsize = 20)
        plt.title(sensor + ' '+str(LONGUEUR)+ 'm patch')

In [ ]:
dict_calibration_sensor = {}
for sensor in dict_sensor :
    dict_calibration_IR = {}
    for k,IR in enumerate(dict_IR) :
        calibration = dict_rms[sensor]['RMS'][k]
        print(sensor,IR,"valeur de calibration :",calibration)
        IR_calibration = dict_IR[IR]["dict_windows_param"][sensor]["all_band"] + calibration
        dict_calibration_IR[IR] = IR_calibration
        dict_IR[IR]["dict_windows_param"][sensor]["calibration"] = dict_calibration_IR
        dict_IR[IR]["dict_windows_param"][sensor]["calibration_value"] = calibration

In [ ]:
dict_IR["IR_17H27"]["dict_windows_param"]["S9"]["calibration_value"]

In [ ]:
dict_IR[IR]["dict_windows"][sensor]

In [ ]:
pseudo_temp_IR["IR_6H55"]['time'] = pd.Timestamp('2019-04-14 07:00:00')
pseudo_temp_IR["IR_7H29"]['time'] = pd.Timestamp('2019-04-14 07:30:00')
pseudo_temp_IR["IR_8H22"]['time'] = pd.Timestamp('2019-04-14 08:30:00')
pseudo_temp_IR["IR_9H28"]['time'] = pd.Timestamp('2019-04-14 09:30:00')
pseudo_temp_IR["IR_10H22"]['time'] = pd.Timestamp('2019-04-14 10:30:00')
pseudo_temp_IR["IR_11H27"]['time'] = pd.Timestamp('2019-04-14 11:30:00')
pseudo_temp_IR["IR_12H31"]['time'] = pd.Timestamp('2019-04-14 12:30:00')
pseudo_temp_IR["IR_13H26"]['time'] = pd.Timestamp('2019-04-14 13:30:00')
pseudo_temp_IR["IR_15H59"]['time'] = pd.Timestamp('2019-04-14 16:00:00')
pseudo_temp_IR["IR_17H27"]['time'] = pd.Timestamp('2019-04-14 17:30:00')

## Moyenne 4 pts

In [ ]:
filetif = ['6H55','7H29','8H22','9H28','10H22','11H27','12H31','13H26',
                               '15H59','17H27']
ls_path_tif,filetif = get_tif(filetif)

dict_value_IR = {}

for i,IR in enumerate(dict_IR) :
    dict_value_target = {}
    with rio.open(ls_path_tif[i]) as dataset : # ouverture du fichier obligatoire pour recup la valeur
        for target in dict_IR[IR]["dict_windows"] :

            hop = dict_IR[IR]["dict_windows"][target]
            mid_indice = (dict_IR[IR]["dict_windows_param"][target]['win'].width) # les indices commencent à 0 !

            mid_x = hop.iloc[len(hop)//2]["x"] 
            mid_y = hop.iloc[(mid_indice//2)-1]["y"]# la valeur du milieu est //2 

            #list des valeurs autour de [0, 2pi] , mouvement anti-horaire

            theta = [0,np.pi/2,np.pi,np.pi * (3/2)]
            r = 1
            

            # allocation des matrices 
            ls_x = [] ; ls_y = []
            print(target)
            print("mid_x :",mid_x,"mid_y :",mid_y)

            for THETA in theta :
                x = mid_x + r*np.cos(THETA)
                y = mid_y + r*np.sin(THETA)
                print("x :",x,"y :",y)
                ls_x.append(x) ; ls_y.append(y)


            with rio.open(ls_path_tif[i]) as dataset : # ouverture du fichier obligatoire pour recup la valeur
                print(ls_path_tif[i])

                dict_IR[IR]["dict_windows_param"][target]['win'].width # résolution -> nb de pixel == nb d'indice 

                ls_value = []
                for k in range(len(ls_x)) :

                    for value in dataset.sample([(float(ls_x[k]),float(ls_y[k]))]) : 
                        ls_value.append(float(value))

                dict_value_target[target] = pd.DataFrame(np.array([ls_x,ls_y,ls_value],dtype = float).T,columns = 
                                                         ["x","y","value"])
    dict_value_IR[IR] = dict_value_target
                
                    


In [ ]:
ls_path_tif,filetif = get_tif(filetif)

dict_value_IR = {}

for i,IR in enumerate(dict_IR) :
    dict_value_target = {}
    with rio.open(ls_path_tif[i]) as dataset : # ouverture du fichier obligatoire pour recup la valeur
        for target in dict_IR[IR]["dict_windows"] :
            
            res = dict_IR[IR]["dict_windows_param"][target]["resolution"] # resolution [x,y]
            hop = dict_IR[IR]["dict_windows"][target]
            mid_indice = (dict_IR[IR]["dict_windows_param"][target]['win'].width) # les indices commencent à 0 !

            mid_x = hop.iloc[len(hop)//2]["x"] 
            mid_y = hop.iloc[(mid_indice//2)-1]["y"]# la valeur du milieu est //2 

            #list des valeurs autour de [0, 2pi] , mouvement anti-horaire
            theta = np.linspace(0,np.pi*2,round(np.pi*2/res[0]))
            ls_x = np.empty([1]) ; ls_y = np.empty([1])
            
            #recherche de points entre deux rayon r1 et r2
            r1 = 0.7 ; r2 = 1.5
            for THETA in theta :
                #
                x1 = mid_x + r1*np.cos(THETA)
                y1 = mid_y + r1*np.sin(THETA)
                
                x2 = mid_x + r2*np.cos(THETA)
                y2 = mid_y + r2*np.sin(THETA)

                ls_x = np.append(ls_x,np.linspace(x1,x2,round((r2-r1)/res[0]))) 
                ls_y = np.append(ls_y,np.linspace(y1,y2,round((r2-r1)/res[1])))

            ls_x = np.delete(ls_x,0) ; ls_y = np.delete(ls_y,0)


            with rio.open(ls_path_tif[i]) as dataset : # ouverture du fichier obligatoire pour recup la valeur
                

                dict_IR[IR]["dict_windows_param"][target]['win'].width # résolution -> nb de pixel == nb d'indice 

                ls_value = []
                for k in range(len(ls_x)) :

                    for value in dataset.sample([(float(ls_x[k]),float(ls_y[k]))]) : 
                        ls_value.append(float(value))

                dict_value_target[target] = pd.DataFrame(np.array([ls_x,ls_y,ls_value],dtype = float).T,columns = 
                                                         ["x","y","value"])
    print(IR)
    dict_value_IR[IR] = dict_value_target
    print("nombre de coordonnées utilisée pour la moyenne :",dict_value_IR[IR][target].shape[0])

In [ ]:
S1 = pd.DataFrame(columns = ["x","y","origin band","band 1","band N","IR"])
S2 = pd.DataFrame(columns = ["x","y","origin band","band 1","band N","IR"])
S3_AIR = pd.DataFrame(columns = ["x","y","origin band","band 1","band N","IR"])
S4 = pd.DataFrame(columns = ["x","y","origin band","band 1","band N","IR"])
S5 = pd.DataFrame(columns = ["x","y","origin band","band 1","band N","IR"])
S6 = pd.DataFrame(columns = ["x","y","origin band","band 1","band N","IR"])
S7 = pd.DataFrame(columns = ["x","y","origin band","band 1","band N","IR"])
S8 = pd.DataFrame(columns = ["x","y","origin band","band 1","band N","IR"])
S9 = pd.DataFrame(columns = ["x","y","origin band","band 1","band N","IR"])
S10 = pd.DataFrame(columns = ["x","y","origin band","band 1","band N","IR"])
dict_sensor = {"S1" : S1,"S2" : S2,"S3-AIR" : S3_AIR,"S4" : S4,"S5" : S5,
            "S6" : S6,"S7" : S7,"S8" : S8,"S9" : S9,"S10" : S10}

BAND_MIN_N = 0
BAND_MAX_N = 0
for sensor in dict_sensor :

    for target in dict_IR :
        min_= dict_IR[target]["dict_windows"][sensor]["band 1"].min()
        max_= dict_IR[target]["dict_windows"][sensor]["band 1"].max()
        if min_ < BAND_MIN_N :
            BAND_MIN_N = min_
            
            
        if max_ > BAND_MAX_N :
            BAND_MAX_N = max_
            

print("BAND_MIN_N",BAND_MIN_N) # valeur minimale retenue pour la normalisation
print("BAND_MAX_N",BAND_MAX_N) # valeur maximale retenue pour la normalisation

for sensor in dict_sensor :
    k = 0 
    
    for target in dict_IR :

        band_shape = dict_IR[target]["dict_windows"][sensor][["band 1"]].shape[0]

        band_min = dict_IR[target]["dict_windows"][sensor]["band 1"].min()
        band_max = dict_IR[target]["dict_windows"][sensor]["band 1"].max()
        
        x = pd.DataFrame(dict_IR[target]["dict_windows"][sensor]["x"])
        y = pd.DataFrame(dict_IR[target]["dict_windows"][sensor]["y"])
        # band originale, sans normalisation
        origin_band = pd.DataFrame(dict_IR[target]["dict_windows"][sensor]["band 1"])
        origin_band.columns = ["origin band"]
        
        # band avec normalisation différente pour chaque IR
        band     = pd.DataFrame((dict_IR[target]["dict_windows"][sensor][["band 1"]]-band_min)/(band_max-band_min))
        
        # band avec normalisation commune pour chaque IR
        band_N   = pd.DataFrame((dict_IR[target]["dict_windows"][sensor][["band 1"]]-BAND_MIN_N)/(BAND_MAX_N-BAND_MIN_N)) 
        band_N.columns = ['band N']

        # Concaténation de plusieurs bandes
        cnte = pd.concat([x,y,origin_band,band,band_N],axis=1)
        dict_sensor[sensor]=dict_sensor[sensor].append(cnte,ignore_index=True)
        
        # Repetition du nom de IR pour chaque valeurs -> boxplot
        dict_sensor[sensor]["IR"][k:(k+band_shape)]  = target
        
        k+=band_shape
    print("completed :",sensor)

In [ ]:
dict_value_IR[IR][sensor].iloc[0]["x"]

In [ ]:
len(dict_value_IR[IR][sensor]["value"])

In [ ]:
dict_value_IR['IR_6H55']["S1"]["x"].iloc[-1]

In [ ]:
TARGET

In [ ]:
X-c_2

In [ ]:
TARGET = sensor_coord.loc[sensor_coord["SensorName"]==sensor]
c_1 = dict_value_IR['IR_6H55']["S1"]["x"].iloc[0]
c_2 = dict_value_IR['IR_6H55']["S1"]["x"].iloc[-1]
X = np.array(TARGET["x"],dtype = float)
Y = np.array(TARGET["y"],dtype=float)
print(X,Y)
circle1 = plt.Circle((X,Y),(c_1-X),color='b', fill=False)
circle2 = plt.Circle((X,Y),(c_2-X),color='r', fill=False)
ax = plt.gca()
ax.set_xlim((X-3,X+3 ))
ax.set_ylim((Y-3,Y+3 ))
ax.add_patch(circle1)
ax.add_patch(circle2)


In [ ]:
# Plot des points choisi pour la pseudo-temp
for k in range( len(dict_value_IR[IR][sensor]["value"])) :
    plt.plot(dict_value_IR[IR][sensor]["x"][k],dict_value_IR[IR][sensor]["y"][k],'ro')

In [ ]:
ls_time

In [ ]:
res = dict_IR['IR_17H27']["dict_windows_param"]["S3-AIR"]["resolution"]

In [ ]:
r1 = 1 ; r2 = 2
ls_x1 = [] ; ls_y1 = []
ls_x2 = [] ; ls_y2 = []
for THETA in theta :
    x1 = mid_x + r1*np.cos(THETA)
    y1 = mid_y + r1*np.sin(THETA)
    ls_x1.append(x1) ; ls_y1.append(y1)
    x2 = mid_x + r2*np.cos(THETA)
    y2 = mid_y + r2*np.sin(THETA)
    ls_x2.append(x2) ; ls_y2.append(y2)

In [ ]:
r1 = 1 ; r2 = 2
liste_x = np.empty([1]) ; liste_y = np.empty([1])
for THETA in theta :
    x1 = mid_x + r1*np.cos(THETA)
    y1 = mid_y + r1*np.sin(THETA)
    x2 = mid_x + r2*np.cos(THETA)
    y2 = mid_y + r2*np.sin(THETA)
    
    liste_x = np.append(liste_x,np.linspace(x1,x2,round((r2-r1)/res[0]))) 
    liste_y = np.append(liste_y,np.linspace(y1,y2,round((r2-r1)/res[1])))

liste_x = np.delete(liste_x,0) ; liste_y = np.delete(liste_y,0)

# Plot d'une sonde + Pseudo Temp° 

In [ ]:
from matplotlib.gridspec import GridSpec
import matplotlib.dates as mdates
requested_sensor = ["S10"]

fig = plt.figure(figsize=(25,15))


SMALL_SIZE = 10
MEDIUM_SIZE = 12
BIGGER_SIZE = 15

plt.rc('font', size=SMALL_SIZE)          # controls default text sizes
plt.rc('axes', titlesize=BIGGER_SIZE)     # fontsize of the axes title !!! 
plt.rc('axes', labelsize=SMALL_SIZE)    # fontsize of the x and y labels
plt.rc('xtick', labelsize=SMALL_SIZE)    # fontsize of the tick labels
plt.rc('ytick', labelsize=MEDIUM_SIZE)    # fontsize of the tick labels
plt.rc('legend', fontsize=SMALL_SIZE)    # legend fontsize
plt.rc('figure', titlesize=BIGGER_SIZE)  # fontsize of the figure title


gs = GridSpec(2, 10, figure=fig)
gs.update(wspace = 0.5, hspace=0.0) # set the spacing between axes.
ax = fig.add_subplot(gs[0, :])

for sensor in requested_sensor :
    
    ls_values = [] # listes des valeurs
    ls_time   = [] # listes des temps 
    ls_var    = [] # listes des écarts-types
    
    print(sensor)
    for IR in dict_value_IR :
        ls_values.append(pseudo_temp[sensor][IR]["values"])
        ls_time.append(pseudo_temp[sensor][IR]["time"])
        ls_var.append(pseudo_temp[sensor][IR]["var"])
        
    # [ val - sigma , val + sigma]
    ls_sigma = [np.array(ls_values)-np.array(ls_var),np.array(ls_values)+np.array(ls_var)]
    SLICE = dict_temp_sensor[sensor]
   
    # Plot de la température des sondes
    plottingtemp_single_label_IR_a(SLICE,fig,ax,'T4',200,6)
    ax.set_ylim(-5,30)
    ax3 = ax.twinx() # instantiate a second axes that shares the same x-axis
    color = "tab:red"
    ax.set_ylim(-5,30)
    ax.grid()
    ax.legend()

    
    # Plot de la pseudo Température 
    ax3.plot(ls_time,ls_values,color = color)
    ax3.plot(ls_time,ls_values,'ro',color = color)
    ax3.tick_params(axis = 'y', labelcolor = color)
    ax3.set_ylabel("Pseudo Température", color = color, fontsize = 20)
    ax3.set_ylim(-5,30)
    
    for w in range(len(ls_time)) :
        ax3.text(ls_time[w],ls_values[w]+0.5,str(ls_var[w])[0:5],fontsize = 15) # valeurs d'écart type
       
    
    # Interval de confiance +/- sigma 
    ax3.plot(ls_time,ls_sigma[0],'m',label = '- sigma') # value - sigma
    ax3.plot(ls_time,ls_sigma[1],'m',label = '+ sigma') # value + sigma
    ax3.legend(loc='upper right', bbox_to_anchor=(1, 0.9))


    # plot des patchs IRs
    for k,IR in enumerate(dict_IR) :
        ax2 = fig.add_subplot(gs[1, k])
        
        window_IR = dict_IR[IR]["dict_windows_param"][requested_sensor[0]]["all_band"]
        window_transform_IR = dict_IR[IR]["dict_windows_param"][requested_sensor[0]]["win_transform"]
        extent = plotting_extent(window_IR,window_transform_IR)
        
        sub_fig = ax2.imshow(window_IR,extent = extent,cmap ='viridis')
        ax2.set_title(IR,fontsize = 20,pad = 20)
        fig.colorbar(sub_fig,ax=ax2,fraction=0.046, pad=0.04)
        
        # Plot des targets
        TARGET = sensor_coord.loc[sensor_coord["SensorName"]==sensor]
        X = np.array(TARGET["x"],dtype = float)
        Y = np.array(TARGET["y"],dtype=float)
        ax2.plot(X,Y,'r*')
        
        # Plot des points choisi pour la pseudo-temp
        for k in range( len(dict_value_IR[IR][sensor]["value"])) :
            ax2.plot(dict_value_IR[IR][sensor]["x"][k],dict_value_IR[IR][sensor]["y"][k],'ro')

plt.title(sensor,fontsize = 20)
ax.set_xticklabels([str(time.hour)+':'+str(time.minute) for time in SLICE['Time']][::8])
ax.set_xlabel('Time' + ' 2019-04-19' )



In [ ]:
dict_sensor[sensor]

### Boxplot + Fenêtre clustering sur données "calibrées"

In [ ]:
dict_sensor[sensor]

### Ajout d'un polynome de degrée n = 6

In [ ]:
from matplotlib.gridspec import GridSpec
requested_sensor = ["S10"]

fig = plt.figure(figsize=(25,15))


SMALL_SIZE = 10
MEDIUM_SIZE = 12
BIGGER_SIZE = 15

plt.rc('font', size=SMALL_SIZE)          # controls default text sizes
plt.rc('axes', titlesize=BIGGER_SIZE)     # fontsize of the axes title !!! 
plt.rc('axes', labelsize=SMALL_SIZE)    # fontsize of the x and y labels
plt.rc('xtick', labelsize=SMALL_SIZE)    # fontsize of the tick labels
plt.rc('ytick', labelsize=BIGGER_SIZE)    # fontsize of the tick labels
plt.rc('legend', fontsize=SMALL_SIZE)    # legend fontsize
plt.rc('figure', titlesize=BIGGER_SIZE)  # fontsize of the figure title


gs = GridSpec(2, 10, figure=fig)
gs.update(wspace = 0.5, hspace=0.0) # set the spacing between axes.
ax = fig.add_subplot(gs[0, :])

for sensor in requested_sensor :
    
    ls_values = [] # listes des valeurs
    ls_time   = [] # listes des temps 
    ls_var    = [] # listes des écarts-types
    
    print(sensor)
    for IR in dict_value_IR :
        ls_values.append(pseudo_temp[sensor][IR]["values"])
        ls_time.append(pseudo_temp[sensor][IR]["time"])
        ls_var.append(pseudo_temp[sensor][IR]["var"])
        
    # [ val - sigma , val + sigma]
    ls_sigma = [np.array(ls_values)-np.array(ls_var),np.array(ls_values)+np.array(ls_var)]
    SLICE = dict_temp_sensor[sensor]
    
     
    # Plot de la température des sondes 
    plottingtemp_single_label_IR_a(SLICE,fig,ax,'T4',200,6)
    ax.set_ylim(-5,30)
    ax3 = ax.twinx() # instantiate a second axes that shares the same x-axis
    color = "tab:red"
    ax.set_ylim(-5,30)
    ax.grid()

    # Plot de la pseudo Température 
    ax3.plot(ls_time,ls_values,color = color)
    ax3.plot(ls_time,ls_values,'ro',color = color)
    ax3.tick_params(axis = 'y', labelcolor = color)
    ax3.set_ylabel("Pseudo Température", color = color, fontsize = 20)
    ax3.set_ylim(-5,30)
    
    
    for w in range(len(ls_time)) :
        ax3.text(ls_time[w],ls_values[w]+0.5,str(ls_var[w])[0:5],fontsize = 15) # valeurs d'écart type
        
       
    
    # Interval de confiance +/- sigma 
    ax3.plot(ls_time,ls_sigma[0],'m',label = '- sigma') # value - sigma
    ax3.plot(ls_time,ls_sigma[1],'m',label = '+ sigma') # value + sigma
    ax3.legend(loc='upper right', bbox_to_anchor=(1, 0.9))
    
        # plot des patchs IRs
    for k,IR in enumerate(dict_IR) :
        ax2 = fig.add_subplot(gs[1, k])
        
        window_IR = dict_IR[IR]["dict_windows_param"][requested_sensor[0]]["all_band"]
        window_transform_IR = dict_IR[IR]["dict_windows_param"][requested_sensor[0]]["win_transform"]
        extent = plotting_extent(window_IR,window_transform_IR)
        
        sub_fig = ax2.imshow(window_IR,extent = extent,cmap ='viridis')
        ax2.set_title(IR,fontsize = 20,pad = 20)
        fig.colorbar(sub_fig,ax=ax2,fraction=0.046, pad=0.04)
        
        # Plot des targets
        TARGET = sensor_coord.loc[sensor_coord["SensorName"]==sensor]
        X = np.array(TARGET["x"],dtype = float)
        Y = np.array(TARGET["y"],dtype=float)
        ax2.plot(X,Y,'r*')
        
        # Plot des points choisi pour la pseudo-temp
        for k in range( len(dict_value_IR[IR][sensor]["value"])) :
            ax2.plot(dict_value_IR[IR][sensor]["x"][k],dict_value_IR[IR][sensor]["y"][k],'ro')
    
    # Ajout d'un polynome de degrée 6
    colors = ['forestgreen', 'green', 'lime']
    lw = 2
    X = np.array(SLICE["Time"]).reshape(-1,1)
    y = np.array(SLICE["T4"]).reshape(-1,1)
    for count, degree in enumerate([5]):
        model = make_pipeline(PolynomialFeatures(degree), Ridge())
        model.fit(X, y)
        y_plot = model.predict(X)
        ax.plot(X, y_plot, color=colors[count], linewidth=lw,
                 label="degree %d" % degree)
        ax.legend(loc='upper right', bbox_to_anchor=(1, 1))
        
plt.title(sensor,fontsize = 20)
ax.set_xticklabels([str(time.hour)+':'+str(time.minute) for time in SLICE['Time']][::8])
ax.set_xlabel('Time' + ' 2019-04-19' )



## Ajout de la RMSE/RMS entre la Pseudo-Temp° et la T° du sensor, pas avec la Pseudo-Temp° interpolée

In [ ]:
r = ["S1","S2","S3-AIR","S4","S5","S6","S7","S8","S9","S10"]

In [ ]:
from matplotlib.gridspec import GridSpec
from sklearn.metrics import mean_squared_error
requested_sensor = ["S1","S2","S3-AIR","S4","S5","S6","S7","S8","S9","S10"]


for sensor in requested_sensor :
    
    # Propriétés graphiques
    fig = plt.figure(figsize=(25,15))


    SMALL_SIZE = 10
    MEDIUM_SIZE = 12
    BIGGER_SIZE = 15

    plt.rc('font', size=BIGGER_SIZE)          # controls default text sizes
    plt.rc('axes', titlesize=BIGGER_SIZE)     # fontsize of the axes title !!! 
    plt.rc('axes', labelsize=BIGGER_SIZE)    # fontsize of the x and y labels
    plt.rc('xtick', labelsize=BIGGER_SIZE)    # fontsize of the tick labels
    plt.rc('ytick', labelsize=BIGGER_SIZE)    # fontsize of the tick labels
    plt.rc('legend', fontsize=BIGGER_SIZE)    # legend fontsize
    plt.rc('figure', titlesize=BIGGER_SIZE)  # fontsize of the figure title

    gs = GridSpec(2, 10, figure=fig)
    gs.update(wspace = 0.5, hspace=0.0) # set the spacing between axes.
    ax = fig.add_subplot(gs[0, :])
    
    ls_values = [] # listes des valeurs
    ls_time   = [] # listes des temps 
    ls_var    = [] # listes des écarts-types
    rms       = [] # listes des RMS
    
    print(sensor)
    for IR in dict_value_IR :
        ls_values.append(pseudo_temp[sensor][IR]["values"])
        ls_time.append(pseudo_temp[sensor][IR]["time"])
        ls_var.append(pseudo_temp[sensor][IR]["var"])
        
    # [ val - sigma , val + sigma]
    ls_sigma = [np.array(ls_values)-np.array(ls_var),np.array(ls_values)+np.array(ls_var)]
    SLICE = dict_temp_sensor[sensor]
    
     
    # Plot de la température des sondes 
    plottingtemp_single_label_IR_a(SLICE,fig,ax,'T4',200,6)
    ax.set_ylim(-5,30)
    ax3 = ax.twinx() # instantiate a second axes that shares the same x-axis
    color = "tab:red"
    ax.set_ylim(-5,30)
    ax.grid()

    # Plot de la pseudo Température 
    ax3.plot(ls_time,ls_values,color = color)
    ax3.plot(ls_time,ls_values,'ro',color = color)
    ax3.tick_params(axis = 'y', labelcolor = color)
    ax3.set_ylabel("Pseudo Température", color = color, fontsize = 20)
    ax3.set_ylim(-5,30)
    
    #Affichage des valeurs de RMS
    for w in range(len(ls_time)) :
        y_actual    = SLICE.loc[SLICE["Time"]==ls_time[w]]["T4"] 
        y_predicted = ls_values[w]
        rms.append(np.absolute(np.mean(y_actual-y_predicted)))
        ax3.text(ls_time[w],ls_values[w]+0.5,str(rms[w])[0:5],fontsize = 15) # valeurs RMS

        
    # Affichage de la valeur de RMSE (une valeur)
    ls_slice_value = []
    for time in ls_time : # compare uniquement les pseudo-temp et les valeurs de sonde 
        ls_slice_value.append(float(SLICE["T4"].loc[SLICE['Time'] == time]))
    
    RMSE = mean_squared_error(ls_slice_value,ls_values,squared=False)
    ax3.text(SLICE["Time"].iloc[-8],28,'RMSE = ' + str(RMSE)[0:5],fontsize = 15) # valeurs RMSE
    
       
    
    # Interval de confiance +/- sigma 
    #ax3.plot(ls_time,ls_sigma[0],'m',label = '- sigma') # value - sigma
    #ax3.plot(ls_time,ls_sigma[1],'m',label = '+ sigma') # value + sigma
    #ax3.legend(loc='upper right', bbox_to_anchor=(1, 0.9))
    
    # plot des patchs IRs
    for k,IR in enumerate(dict_IR) :
        ax2 = fig.add_subplot(gs[1, k])
        
        window_IR = dict_IR[IR]["dict_windows_param"][sensor]["all_band"]
        window_transform_IR = dict_IR[IR]["dict_windows_param"][sensor]["win_transform"]
        extent = plotting_extent(window_IR,window_transform_IR)
        
        sub_fig = ax2.imshow(window_IR,extent = extent,cmap ='viridis')
        ax2.set_title(IR,fontsize = 20,pad = 20)
        fig.colorbar(sub_fig,ax=ax2,fraction=0.046, pad=0.04)
        
        # Plot des targets
        TARGET = sensor_coord.loc[sensor_coord["SensorName"]==sensor]
        X = np.array(TARGET["x"],dtype = float)
        Y = np.array(TARGET["y"],dtype=float)
        ax2.plot(X,Y,'r*')
        
        # Plot des points choisi pour la pseudo-temp
        for k in range( len(dict_value_IR[IR][sensor]["value"])) :
            ax2.plot(dict_value_IR[IR][sensor]["x"][k],dict_value_IR[IR][sensor]["y"][k],'ro')
    
    # Ajout d'un polynome de degrée n
    colors = ['forestgreen', 'green', 'lime']
    lw = 2
    #X = np.array(SLICE["Time"]).reshape(-1,1)
    #y = np.array(SLICE["T4"]).reshape(-1,1)
    X = np.array(ls_time,dtype='datetime64[ns]').reshape(-1,1)
    y = np.array(ls_values).reshape(-1,1)
    for count, degree in enumerate([5]):
        model = make_pipeline(PolynomialFeatures(degree), Ridge())
        model.fit(X, y)
        y_plot = model.predict(X)
        ax.plot(X, y_plot, color=colors[count], linewidth=lw,
                 label="degree %d" % degree)
        ax.legend(loc='upper right', bbox_to_anchor=(1, 1))
        
    ax.set_title(sensor +" et valeurs de RMS pour chaque pseudo température",fontsize = 20)
    ax.set_xticklabels([str(time.hour)+':'+str(time.minute) for time in SLICE['Time']][::8])
    ax.set_xlabel('Time (heures:minutes)' + ' 2019-04-19' )
    fileout =  "C:/Users/Alexandre/Desktop/Cours/Stage/Temp_vs_IR/2m/"+ str(sensor)+".png"
    #fig.savefig(fileout)




In [ ]:
ls_slice_value = []
for time in ls_time :
    print(time)
    ls_slice_value.append(float(SLICE["T4"].loc[SLICE['Time'] == time]))

In [ ]:
mean_squared_error(ls_slice_value,ls_values,squared=False)

In [ ]:
y_actual    = SLICE.loc[SLICE["Time"]==ls_time[w]]["T4"] 
y_predicted = ls_values[w]
        rms.append(mean_squared_error()

In [ ]:
from sklearn.linear_model import Ridge
from sklearn.preprocessing import PolynomialFeatures
from sklearn.pipeline import make_pipeline

colors = ['teal', 'yellowgreen', 'gold']
lw = 2
X = np.array(SLICE["Time"]).reshape(-1,1)
y = np.array(SLICE["T4"]).reshape(-1,1)
for count, degree in enumerate([3, 4, 5]):
    model = make_pipeline(PolynomialFeatures(degree), Ridge())
    model.fit(X, y)
    y_plot = model.predict(X)
    plt.plot(X, y_plot, color=colors[count], linewidth=lw,
             label="degree %d" % degree)

In [ ]:
hop = dict_IR['IR_6H55']["dict_windows"]["S10"]
mid_x = hop.iloc[len(hop)//2]["x"]
mid_y = hop.iloc[76//2 -1]["y"]
theta = [0,np.pi/2,np.pi,np.pi*3/4]
r = 1
ls_x = [] ; ls_y = []
print("mid_x :",mid_x,"mid_y :",mid_y)
for THETA in theta :
    x = mid_x + r*np.cos(THETA)
    y = mid_y + r*np.sin(THETA)
    print("x :",x,"y :",y)
    ls_x.append(x) ; ls_y.append(y)
    
    
    


In [ ]:
#Methode pour discriminer des IRs

request_IR = ['IR_11H27',"IR_12H31"]
request_sensor = ["S9","S2","S4"]
requested_IR = Piren_IR[request_IR]

## Creation de mask pour plusieurs images IR
requested_ls_mask_image, requested_ls_out_transform= IR_mask(requested_IR.loc["IR_src"],requested_shapes,requested_ls_coord_circle)
#Methode pour discriminer des sondes

## creation d'un rayon de taille r autour des sensors
ls_sensor = Sensor_coord["SensorName"] # : toutes les sondes
ls_coord_circle,Shape_to_json,circle_name = circle_sensor(ls_sensor,Sensor_coord)

#Creat a shape in GeoJSON format in order to be read with rio and 
#serve as mask to crop selected area in the shape

shapes,shapes_names = circle_to_shape(ls_coord_circle,Shape_to_json,circle_name)

requested_names  =  []
requested_shapes =  []
requested_ls_coord_circle = []
for i,names in enumerate(shapes_names) :
    for k in range(len(request_sensor)) :  
        if names == request_sensor[k] :
            requested_names.append(names)
            requested_shapes.append(shapes[i])
            requested_ls_coord_circle.append(ls_coord_circle[i])
            
requested_names
requested_shapes
len(requested_ls_coord_circle)

In [ ]:
## 1e moitié des sondes IR, avec une echelle defini par l'utilisateur
lim_min = -10
lim_max = 20
lim_tif = len(Piren_IR.loc["IR_src"])
lim_tif = 4
fig_IR,axtemp=plt.subplots(lim_tif,len(ls_sensor),figsize=(25,25))
for i in range(lim_tif) :
    for k,mask_image in enumerate(ls_mask_image[i]) :
        sub_fig = axtemp[i,k].imshow(mask_image[0],cmap ='viridis')#,vmin = lim_min,vmax = lim_max)
        sub =show(mask_image, transform= ls_out_transform[i],
            ax=axtemp[i,k],title=[ls_sensor[k]+'  '+filetif[i]])#,vmin = lim_min,vmax = lim_max)

        sub_fig_colorbar=fig_IR.colorbar(sub_fig,ax = axtemp[i,k])
        sub_fig_colorbar.set_label('T(°C)', rotation=0)

In [ ]:
## Chercher si une variable existe ou pas
B = 5
try: 
    B
except NameError:
        A = 'AHA'
print(A)

In [ ]:
## Discrimination des IRs
Piren_IR[request].loc["IR_src"]
len(ls_mask_image_requested)

In [ ]:
for i in range(len(Piren_IR.loc["IR_src"])):
    print(Piren_IR.loc["IR_src"][i])
print("nombre d'image IR :",len(Piren_IR.loc["IR_src"]))
               

In [ ]:
## 1e moitié des sondes IR, avec une echelle defini par l'utilisateur
lim_min = -10
lim_max = 20
len_IR = len(requested_IR)
len_sondes =
fig_IR,axtemp=plt.subplots(lim_tif,len(ls_sensor),figsize=(25,25))
for i in range(lim_tif) :
    for k,mask_image in enumerate(ls_mask_image[i]) :
        sub_fig = axtemp[i,k].imshow(mask_image[0],cmap ='viridis')#,vmin = lim_min,vmax = lim_max)
        sub =show(mask_image, transform= ls_out_transform[i],
            ax=axtemp[i,k],title=[ls_sensor[k]+'  '+filetif[i]])#,vmin = lim_min,vmax = lim_max)

        sub_fig_colorbar=fig_IR.colorbar(sub_fig,ax = axtemp[i,k])
        sub_fig_colorbar.set_label('T(°C)', rotation=0)

In [ ]:
## plot de tous les histogram autours des sondes 

fig2,(axhist)=plt.subplots(9,figsize=(10,30),sharex=True, sharey=True)
for k,mask_image in enumerate(ls_mask_image) :
    show_hist(mask_image, bins=50, histtype='stepfilled', lw=0.0, stacked=False, ax=axhist[k], alpha=0.3, title=circle_name[k][k])


In [ ]:
## Plot de chaque couple histo/image

for k,mask_image in enumerate(ls_mask_image) :
    fig2,(axhist,axrgb)=plt.subplots(1,2,figsize=(50,50))
    show_hist(mask_image, bins=50, histtype='stepfilled', lw=0.0, stacked=False, ax=axhist, alpha=0.3, title=circle_name[k][k])
    show(mask_image, transform=ls_out_transform,ax=axrgb,title=circle_name[k][k])
    #plt.plot(np.array(Sensor_coord["x"][k]),np.array(Sensor_coord["y"][k]),'bo')

In [ ]:
for k in range(1,len(Piren_IR_ls)) :
    Piren_IR.assign(locals()[IR_%s % Piren_IR_name[1][k]] = Piren_IR_ls[k])

In [ ]:
for k in range(1,len(filetif)) :
    locals()['SS_%s' % filetif[k]] = 1
    print(locals()['SS_%s' % filetif[k]])

locals()['SS_%s' % filetif[1]]

In [ ]:
out_meta = IR_src.meta
for k in range(len(list_coord_circle)) :
    out_image, out_transform = rio.mask.mask(IR_src, shapes[k], crop=True, filled=False)
    """
    And update height and width of cropped image with its meta data"""
    out_meta.update({"driver": "GTiff",
             "height": out_image.shape[1],
             "width": out_image.shape[2],
             "transform": out_transform})
    
    fig2,(axhist, axrgb)=plt.subplots(1,2, figsize=(21,7))
    show(out_image, transform=out_transform,ax=axrgb)
    #plt.plot(np.array(Sensor_coord["x"][k]),np.array(Sensor_coord["y"][k]),'bo')
    show_hist(out_image, bins=50, histtype='stepfilled', lw=0.0, stacked=False, ax=axhist, alpha=0.3, title=circle_name[k][k])

In [ ]:
out_meta = IR_src.meta
out_image, out_transform = rio.mask.mask(IR_src, shapes[0], crop=True, filled=False)
"""
And update height and width of cropped image with its meta data"""
out_meta.update({"driver": "GTiff",
         "height": out_image.shape[1],
         "width": out_image.shape[2],
         "transform": out_transform})

show(out_image, transform=out_transform)

In [ ]:
#Rasterio 
IR_src = rio.open(os.path.join(ls_path_tif[0]))
#show((src,1)) # Affiche la premiere bande
#fig1,ax1 = show((src,1),cmap='viridis')
Piren_IR_array=IR_src.read(1) # Lit la bande 1
Piren_Limits = plotting_extent(IR_src) # Limites
Piren_Limits
Piren_res = IR_src.res # resolution
Piren_res
fig_IR,ax_IR=plt.subplots(1,figsize=(50,50))
sub_fig=ax_IR.imshow(Piren_IR_array,extent=Piren_Limits,cmap='viridis')
ax_IR.set_xlabel('x')
ax_IR.set_ylabel('y')
ax_IR.set_label(ls_path_tif[0])
sub_fig_colorbar=fig_IR.colorbar(sub_fig)
sub_fig_colorbar.set_label('T(°C)', rotation=0, labelpad=-28, y=1.05)

## Plot des coordonnees de la sonde
#plt.plot(np.array(Sensor_coord["x"]),np.array(Sensor_coord["y"]),'bo')
## Plot des coordonnees de la sonde
#ax_IR.annotate(str(sensor_GPS["SensorName"]),[np.array(Sensor_coord["x"]),np.array(Sensor_coord["y"])])

In [ ]:
Piren_IR_test = pd.DataFrame(np.array([Piren_IR[0][0],Piren_IR[0][1],Piren_IR[0][2]],dtype=object).T,index =
                            ["IR_array","Limits","IR_src"],columns = ['001'])
Piren_IR_test['001']['IR_array']

In [ ]:
sss = np.array([Piren_IR_array,Piren_Limits,IR_src],dtype=object).T
sss.shape

In [ ]:
#Creat a shape in GeoJSON format in order to be read with rio and 
#serve as mask to crop selected area in the shape
shapes = []
for j in range(len(list_coord_circle)):
    with fiona.open(Shape_to_json[j],'r') as Test_image:
        #print(list(Test_image)) # On va chercher à chopper la propriete Geometry
        shapes.append([feature["geometry"] for feature in Test_image])
print(len(shapes))

In [ ]:
len_coord = len(Sensor_coord)
value = []
for k in range(0,len_coord) :
    for val in IR_src.sample([(float(Sensor_coord["x"][k]),float(Sensor_coord["y"][k]))]): 
        value.append(val)
    coord_value = np.array([Sensor_coord["x"],Sensor_coord["y"],value])
coord_value.shape

In [ ]:
Piren_IR_array.shape
IR_src.tags
aaa = rio.sample.sample_gen(IR_src,([523628.71,5366296.81]),indexes=None,masked=False)
for val in IR_src.sample([(float(Sensor_coord["x"][0]),float(Sensor_coord["y"][0]))]): 
    print(val)

In [ ]:
#Recupere les noms des IR dans le dossier path, 
#par defaut = './traitement_PIREN/'
ls_path_tif = path_IR()
ls_path_tif[0][0]
for test_tif in ls_path_tif:
    print(test_tif[0],type(test_tif[0]))
    if test_tif[0].find('003') > 0:
        aaa = test_tif[0]
        break
aaa

In [ ]:
# Ouverture et recupération des positions de Sensors 
Sensor_Name_File_GPS=[]
Sensor_x=[]
Sensor_y=[]
k = 0
with open("./traitement_PIREN/sondes_gps_UTM31N_phase1.txt") as File_GPS:
    csvReader=csv.reader(File_GPS, delimiter='\t')
    for row in csvReader:
        Sensor_Name_File_GPS.append(row[0]) ## colonne nom du fichier
        # str list to float list, for plot option
        Sensor_x.append(float(row[1])) # colonne coordonnees x 
        Sensor_y.append(float(row[2])) # colonne coordonnees y
        print("x =",Sensor_x[k],'  y =',Sensor_y[k])
        k+=1

In [ ]:
### Recupere les noms des Sensors dans le dossier path, par defaut = './Data/'
path = './traitement_PIREN/'
ls_path_tif = []
ls_path = os.listdir(path=path)
for tif in ls_path:
    if tif.find('.tif') > 0 :
        ls_path_tif.append([path+tif])

In [ ]:
Sensor_coord =pd.DataFrame(np.array([Sensor_Name_File_GPS,Sensor_x,Sensor_y]).T)
Sensor_coord

In [ ]:
## creation d'un rayon de taille r autour d une Sensor (S10)
list_coord_circle = []
Shape_to_json=[]
circle_name = []

len_sensor = len(Sensor_coord['SensorName'])
for SensorName in Sensor_coord['SensorName'] :
    sensor_GPS = Sensor_coord.loc[Sensor_coord['SensorName']==str(SensorName)]
    center = Point(sensor_GPS["x"],sensor_GPS["y"])
    circle = center.buffer(1)
    #Val de chaque extremitees du cercle
    x_circle,y_circle = circle.exterior.xy
    list_coord_circle.append([np.array(x_circle),np.array(y_circle)])
    # Transfo des donnees en Geoseries : 
    #Json -> https://fr.wikipedia.org/wiki/JavaScript_Object_Notation
    # Format qui contient toutes les proprietes + points ext du cercle
    Shape_to_json.append(gpd.GeoSeries([circle]).to_json())
    circle_name.append(sensor_GPS['SensorName'])
    list_coord_circle

In [ ]:
plot_test = gpd.GeoSeries([circle])
plot_test.head()

## Ultérieur

In [ ]:
## creation d'un rayon de taille r autour des sensors
ls_sensor = Sensor_coord["SensorName"] # : toutes les sondes
ls_coord_circle,Shape_to_json,circle_name = circle_sensor(ls_sensor,Sensor_coord)

#Creat a shape in GeoJSON format in order to be read with rio and 
#serve as mask to crop selected area in the shape

shapes,shapes_names = circle_to_shape(ls_coord_circle,Shape_to_json,circle_name)

## Creation de mask pour plusieurs images IR
ls_mask_image, ls_out_transform= IR_mask(Piren_IR.loc["IR_src"],shapes,ls_coord_circle)


In [ ]:
## plot toutes les sondes pour ttes IR, avec une echelle defini par l'utilisateur
lim_min = -20
lim_max = 40
fig_IR,axtemp=plt.subplots(10,10,figsize=(40,40),sharex=True, sharey=True)

SMALL_SIZE = 8
MEDIUM_SIZE = 15
BIGGER_SIZE = 20

plt.rc('font', size=SMALL_SIZE)          # controls default text sizes
plt.rc('axes', titlesize=BIGGER_SIZE)     # fontsize of the axes title !!! 
plt.rc('axes', labelsize=SMALL_SIZE)    # fontsize of the x and y labels
plt.rc('xtick', labelsize=SMALL_SIZE)    # fontsize of the tick labels
plt.rc('ytick', labelsize=MEDIUM_SIZE)    # fontsize of the tick labels
plt.rc('legend', fontsize=SMALL_SIZE)    # legend fontsize
plt.rc('figure', titlesize=BIGGER_SIZE)  # fontsize of the figure title

for i in range(len(Piren_IR.loc["IR_src"])) :
    for k,mask_image in enumerate(ls_mask_image[i]) :
        sub_fig = axtemp[i,k].imshow(mask_image[0],cmap ='viridis',vmin = lim_min,vmax = lim_max)
        sub =show(mask_image, transform= ls_out_transform[i],vmin = lim_min,vmax = lim_max,
            ax=axtemp[i,k],title=[circle_name[k][k]+'  '+filetif[i]])

sub_fig_colorbar=fig_IR.colorbar(sub_fig,ax = axtemp[0:10])
sub_fig_colorbar.set_label('T(°C)', rotation=0, labelpad=-28, y=1.05)

In [ ]:
for IR in Piren_IR :
        print(IR)

## echantillon de sonde / IR

In [ ]:
#Methode pour discriminer des IRs

request_IR = ['IR_11H27']
request_sensor = ["S9"]
requested_IR = Piren_IR[request_IR]


## creation d'un rayon de taille r autour des sensors
ls_sensor = Sensor_coord["SensorName"] # : toutes les sondes
ls_coord_circle,Shape_to_json,circle_name = circle_sensor(ls_sensor,Sensor_coord)

#Creat a shape in GeoJSON format in order to be read with rio and 
#serve as mask to crop selected area in the shape

shapes,shapes_names = circle_to_shape(ls_coord_circle,Shape_to_json,circle_name)

requested_names  =  []
requested_shapes =  []
requested_ls_coord_circle = []
for i,names in enumerate(shapes_names) :
    for k in range(len(request_sensor)) :  
        if names == request_sensor[k] :
            requested_names.append(names)
            requested_shapes.append(shapes[i])
            requested_ls_coord_circle.append(ls_coord_circle[i])
            
## Creation de mask pour plusieurs images IR
requested_ls_mask_image, requested_ls_out_transform= IR_mask(requested_IR.loc["IR_src"],requested_shapes,requested_ls_coord_circle)
#Methode pour discriminer des sondes

## Plot d'un echantillon i = IR , k = Sondes
lim_min = -10
lim_max = 20
len_IR = len(requested_ls_mask_image)
len_shapes = len(requested_shapes)
step = 1

SMALL_SIZE = 8
MEDIUM_SIZE = 15
BIGGER_SIZE = 20

plt.rc('font', size=SMALL_SIZE)          # controls default text sizes
plt.rc('axes', titlesize=BIGGER_SIZE)     # fontsize of the axes title !!! 
plt.rc('axes', labelsize=SMALL_SIZE)    # fontsize of the x and y labels
plt.rc('xtick', labelsize=SMALL_SIZE)    # fontsize of the tick labels
plt.rc('ytick', labelsize=MEDIUM_SIZE)    # fontsize of the tick labels
plt.rc('legend', fontsize=SMALL_SIZE)    # legend fontsize
plt.rc('figure', titlesize=BIGGER_SIZE)  # fontsize of the figure title


if len_IR == 1 and len_shapes == 1 : 
    
    fig_IR,axtemp=plt.subplots(1,figsize=(25,25))
    
    for i in range(len_IR) :
        for k,mask_image in enumerate(requested_ls_mask_image[i]) :
            sub_fig = axtemp.imshow(mask_image[0],cmap ='viridis')#,vmin = lim_min,vmax = lim_max)
            sub =show(mask_image, transform= requested_ls_out_transform[i],
                ax=axtemp,title=[requested_names[k]+'  '+request_IR[i]])#,vmin = lim_min,vmax = lim_max)

            sub_fig_colorbar=fig_IR.colorbar(sub_fig,ax = axtemp)
            sub_fig_colorbar.set_label('T(°C)', rotation=0)
else :
        
        fig_IR,axtemp=plt.subplots(len_IR,len_shapes,figsize=(25,25))

        for i in range(len_IR) :
            for k,mask_image in enumerate(requested_ls_mask_image[i]) :
                sub_fig = axtemp[i,k].imshow(mask_image[0],cmap ='viridis')#,vmin = lim_min,vmax = lim_max)
                sub =show(mask_image, transform= requested_ls_out_transform[i],
                    ax=axtemp[i,k],title=[requested_names[k]+'  '+request_IR[i]])#,vmin = lim_min,vmax = lim_max)

                sub_fig_colorbar=fig_IR.colorbar(sub_fig,ax = axtemp[i,k])
                sub_fig_colorbar.set_label('T(°C)', rotation=0)

## Comparaison avec les données de la sondes la plus shallow (5cm) a +/-1 h 

In [ ]:
### A MODIFIER 
date_1='2019-04-14 12:30:00'
date_2='2019-04-14 14:30:00'


In [ ]:
### Crée des Tableaux pour chaque sonde selon leur nom : RawS1,RawS2,..
### Et un dictionnaire associee

ls_path = path_sonde()
dict_temp_sensor = {}
dict_slice_temp = {}

for i,ls_path_sonde in enumerate(ls_path):
    name = ls_path_sonde[0][ls_path_sonde[0].find("S"):(ls_path_sonde[0].find(".csv"))]
    Raw = readingtemp(ls_path_sonde[0])
    dict_temp_sensor[name] = Raw
    dict_slice_temp[name] = slice_raw(dict_temp_sensor[name],date_1,date_2)
    print("completed :",name)

    

    


In [ ]:
print(dict_temp_sensor["S9"])

In [ ]:
#Methode pour discriminer des IRs

request_IR = ['IR_6H55']
request_sensor = ["S10"]
SLICE = sliceS10  ## à remplacer 

requested_IR = Piren_IR[request_IR]
## creation d'un rayon de taille r autour des sensors
ls_sensor = Sensor_coord["SensorName"] # : toutes les sondes
ls_coord_circle,Shape_to_json,circle_name = circle_sensor(ls_sensor,Sensor_coord)

#Creat a shape in GeoJSON format in order to be read with rio and 
#serve as mask to crop selected area in the shape
shapes,shapes_names = circle_to_shape(ls_coord_circle,Shape_to_json,circle_name)


requested_names  =  []
requested_shapes =  []
requested_ls_coord_circle = []
for i,names in enumerate(shapes_names) :
    for k in range(len(request_sensor)) :  
        if names == request_sensor[k] :
            requested_names.append(names)
            requested_shapes.append(shapes[i])
            requested_ls_coord_circle.append(ls_coord_circle[i])
            
## Creation de mask pour plusieurs images IR
requested_ls_mask_image, requested_ls_out_transform= IR_mask(requested_IR.loc["IR_src"],requested_shapes,requested_ls_coord_circle)
#Methode pour discriminer des sondes

## Plot d'un echantillon i = IR , k = Sondes
lim_min = -10
lim_max = 20
len_IR = len(requested_ls_mask_image)
len_shapes = len(requested_shapes)
step = 1

font = {'family' : 'DejaVu Sans',
        'weight' : 'bold',
        'size'   : 28 }

plt.rc('font', **font)



fig_IR,axtemp=plt.subplots(2,1,figsize=(25,25))

for i in range(len_IR) :
    for k,mask_image in enumerate(requested_ls_mask_image[i]) :
        sub_fig = axtemp[0].imshow(mask_image[0],cmap ='viridis')#,vmin = lim_min,vmax = lim_max)
        sub =show(mask_image, transform= requested_ls_out_transform[i],
            ax=axtemp[0],title=[requested_names[k]+'  '+request_IR[i]])#,vmin = lim_min,vmax = lim_max)

        sub_fig_colorbar=fig_IR.colorbar(sub_fig,ax = axtemp[0])
        sub_fig_colorbar.set_label('T(°C)', rotation=0)

plottingtemp_single_label_IR(SLICE,fig_IR,axtemp[1],'T4',step)

In [ ]:
tif = "./traitement_PIREN/vis_piren_phase_1_cropped_clustered_7k.tif"
dataset = rio.open(tif)

In [ ]:
dataset.res